In [1]:
# 1. Imports
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import re
import time
import pyautogui
import matplotlib

# import scipy
from scipy import interpolate
from scipy import signal

# import skimage
from skimage import io, filters, exposure, measure, transform
from scipy.signal import find_peaks, savgol_filter
from scipy.optimize import linear_sum_assignment

# 2. Pandas settings
pd.set_option('mode.chained_assignment',None)

# 3. Plot settings
# Here we use this mode because displaying images 
# in new windows is more convenient for this code.
%matplotlib qt 
# To switch back to inline display, use : 
# %matplotlib widget or %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})

SMALLER_SIZE = 8
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# 4. Other settings
# These regex are used to correct the stupid date conversions done by Excel
dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [2]:
SCALE_100X = 15.8 # pix/µm 

## Classes & functions

### Import experiemental data table

Here the Experimental Data Table is imported. This table, filled after each experiment, contains all the relevant infos about experimental conditions, and can be both read by the users (to keep a history of the experiments done) and the computer (to access relevant parameters).

In [3]:
# experimentalDataDir = "C://Users//josep//Desktop//ActinCortexAnalysis//ExperimentalData"
experimentalDataDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//ExperimentalData"

def getExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
            if '.1' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            try:
                if expConditionsDF[col].dtype == 'string':
                    listTextColumns.append(col)
            except:
                aaaa=0
                #Ok

        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        try:
            expConditionsDF['optical index correction'] = \
                      expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                    / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        except:
            print('optical index correction already in ' + str(expConditionsDF['optical index correction'].dtype) + ' type.')

        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        try:
            expConditionsDF['ramp field'] = \
            expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])
        except:
            aaaa=0
            #Ok

        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
#     reorgaList = np.array([i for i in range(len(expConditionsDF.columns))])
#     reorgaList[2] = reorgaList[-1]
#     reorgaList[3:] = reorgaList[3:] - np.ones(len(reorgaList)-3)
#     expConditionsDF = expConditionsDF[expConditionsDF.columns[reorgaList]]
    
    return(expConditionsDF)

expDf = getExperimentalConditions()

expDf.head()

Extracted a table with 53 lines and 24 columns.
optical index correction already in Float64 type.


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments,manipID
0,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,100X,15.8,oil,0.875,1.15,...,0,5,3_40,1s,False,3,500,143_95,<NA>,DEFAULT_DEFAULT
1,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-04_M1
2,20-08-04,M2,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-04_M2
3,20-08-05,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-05_M1
4,20-08-05,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-05_M2


### Utility functions

In [4]:
def findInfosInFileName(f, infoType):
    """
    Return a given type of info from a file name.
    Inputs : f (str), the file name.
             infoType (str), the type of info wanted.
             infoType can be equal to : 
             * 'M', 'P', 'C' -> will return the number of manip (M), well (P), or cell (C) in a cellID.
             ex : if f = '21-01-18_M2_P1_C8.tif' and infoType = 'C', the function will return 8.
             * 'manipID'     -> will return the full manip ID.
             ex : if f = '21-01-18_M2_P1_C8.tif' and infoType = 'manipID', the function will return '21-01-18_M2'.
             * 'cellID'     -> will return the full cell ID.
             ex : if f = '21-01-18_M2_P1_C8.tif' and infoType = 'cellID', the function will return '21-01-18_M2_P1_C8'.
    """
    if infoType in ['M', 'P', 'C']:
        acceptedChar = [str(i) for i in range(10)] + ['.', '-']
        string = '_' + infoType
        iStart = re.search(string, f).end()
        i = iStart
        infoString = '' + f[i]
        while f[i+1] in acceptedChar and i < len(f)-1:
            i += 1
            infoString += f[i]
    
    elif infoType == 'manipID':
        datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", f)
        date = f[datePos.start():datePos.end()]
        manip = 'M' + findInfosInFileName(f, 'M')
        infoString = date + '_' + manip
        
    elif infoType == 'cellID':
        datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", f)
        date = f[datePos.start():datePos.end()]
        infoString = date + '_' + 'M' + findInfosInFileName(f, 'M') + \
                            '_' + 'P' + findInfosInFileName(f, 'P') + \
                            '_' + 'C' + findInfosInFileName(f, 'C')
    
    return(infoString)

def isFileOfInterest(f, manips, wells, cells):
    """
    Determine if a file f correspond to the given criteria.
    More precisely, return a boolean saying if the manip, well and cell number are in the given range.
    f is a file name. Each of the fields 'manips', 'wells', 'cells' can be either a number, a list of numbers, or 'all'.
    Example : if f = '21-01-18_M2_P1_C8.tif'
    * manips = 'all', wells = 'all', cells = 'all' -> the function return True.
    * manips = 1, wells = 'all', cells = 'all' -> the function return False.
    * manips = [1, 2], wells = 'all', cells = 'all' -> the function return True.
    * manips = [1, 2], wells = 2, cells = 'all' -> the function return False.
    * manips = [1, 2], wells = 1, cells = [5, 6, 7, 8] -> the function return True.
    Note : if manips = 'all', the code will consider that wells = 'all', cells = 'all'.
           if wells = 'all', the code will consider that cells = 'all'.
           This means you can add filters only in this order : manips > wells > cells.
    """
    test = False
    if f.endswith(".tif"):
        if manips == 'all':
            test = True
        else:
            try:
                manips_str = [str(i) for i in manips]
            except:
                manips_str = [str(manips)]
            infoM = findInfosInFileName(f, 'M')
            if infoM in manips_str:
                if wells == 'all':
                    test = True
                else:
                    try:
                        wells_str = [str(i) for i in wells]
                    except:
                        wells_str = [str(wells)]
                    infoP = findInfosInFileName(f, 'P')
                    if infoP in wells_str:
                        if cells == 'all':
                            test = True
                        else:
                            try:
                                cells_str = [str(i) for i in cells]
                            except:
                                cells_str = [str(cells)]
                            infoC = findInfosInFileName(f, 'C')
                            if infoC in cells_str:
                                test = True
    return(test)

def compute_cost_matrix(XY1,XY2):
    """
    Compute a custom cost matrix between two arrays of XY positions.
    Here the costs are simply the squared distance between each XY positions.
    Example : M[2,1] is the sqaured distance between XY1[2] and XY2[1], 
    which is ((XY2[1,1]-XY1[2,1])**2 + (XY2[1,0]-XY1[2,0])**2)
    """
    N1, N2 = XY1.shape[0],XY2.shape[0]
    M = np.zeros((N1, N2))
    for i in range(N1):
        for j in range(N2):
            M[i,j] = (np.sum((XY2[j,:] - XY1[i,:]) ** 2))
    return(M)

def ui2array(uixy):
    """
    Translate the output of the function plt.ginput() (which are lists of tuples), in a XY array.
    """
    n = len(uixy)
    XY = np.zeros((n, 2))
    for i in range(n):
        XY[i,0], XY[i,1] = uixy[i][0], uixy[i][1]
    return(XY)

def getROI(roiSize, x0, y0, nx, ny):
    """
    Return coordinates of top left (x1, y1) and bottom right (x2, y2) corner of a ROI, 
    and a boolean validROI that says if the ROI exceed the limit of the image.
    Inputs : 
    - roiSize, the width of the (square) ROI.
    - x0, y0, the position of the central pixel.
    - nx, ny, the size of the image.
    Note : the ROI is done so that the final width (= height) of the ROI will always be an odd number.
    """
    roiSize += roiSize%2
    x1 = int(np.floor(x0) - roiSize*0.5) - 1
    x2 = int(np.floor(x0) + roiSize*0.5)
    y1 = int(np.floor(y0) - roiSize*0.5) - 1
    y2 = int(np.floor(y0) + roiSize*0.5)
    if min([x1,nx-x2,y1,ny-y2]) < 0:
        validROI = False
    else:
        validROI = True
    return(x1, y1, x2, y2, validROI)

def getDepthoCleanSize(D, scale):
    cleanSize = int(np.floor(1*D*scale))
    cleanSize += 1 + cleanSize%2
    return(cleanSize)

def squareDistance_V0(M, V, normalize = False): # MAKE FASTER !!!
    top = time.time()
    n, m = M.shape[0], M.shape[1]
    # len(V) should be m
    result = np.zeros(n)
    if normalize:
        V = V/np.mean(V)
    for i in range(n):
        if normalize:
            Mi = M[i,:]/np.mean(M[i,:])
        else:
            Mi = M[i,:]
        d = np.sum((Mi-V)**2)
        result[i] = d
    print('DistanceCompTime')
    print(time.time()-top)
    return(result)

def squareDistance(M, V, normalize = False): # MAKE FASTER !!!
#     top = time.time()
    n, m = M.shape[0], M.shape[1]
    # len(V) should be m
    if normalize:
        V = V/np.mean(V)
    V = np.array([V])
    MV = np.repeat(V, n, axis = 0)
    if normalize:
        M = (M.T/np.mean(M, axis = 1).T).T
    R = np.sum((M-MV)**2, axis = 1)
#     print('DistanceCompTime')
#     print(time.time()-top)
    return(R)

def assembleDists(listD, listStatus, Nup, NVox):
    N = len(listStatus)
    offsets = np.array(listStatus) - np.ones(N) * (Nup//2 + 1)
    offsets = offsets.astype(int)
    listD2 = []
    for i in range(N):
        if offsets[i] < 0:
            shift = abs(offsets[i])*NVox
            D = listD[i]
            fillVal = D[-1]
            D2 = np.concatenate((D[shift:],fillVal*np.ones(shift))).astype(np.uint16)
            listD2.append(D2)
        if offsets[i] == 0:
            D = listD[i].astype(np.uint16)
            listD2.append(D)
        if offsets[i] > 0:
            shift = abs(offsets[i])*NVox
            D = listD[i]
            fillVal = D[0]
            D2 = np.concatenate((fillVal*np.ones(shift),D[:-shift])).astype(np.uint16)
            listD2.append(D2)
    return(np.array(listD2))

def uiThresholding(I, method = 'otsu', factorT = 0.6, increment = 0.1):
    nz = I.shape[0]
    threshold = factorT*filters.threshold_otsu(I)
    maxVal = np.zeros(nz)
    for z in range(nz):
        maxVal[z] = np.max(I[z])
    z_max = np.argmax(maxVal)
    I_max = I[z_max]
    I_thresh = I_max > threshold
    fig, ax = plt.subplots(1,1)
    ax.imshow(I_thresh, cmap = 'gray')
    fig.show()
    QA = pyautogui.confirm(
                text='Is the threshold {:.0f} satisfying?'.format(threshold),
                title='Confirm threshold', 
                buttons=['Yes', 'Try lower', 'Try higher'])
    plt.close(fig)
    if QA == 'Try lower':
        threshold = uiThresholding(I, method = 'otsu', factorT = factorT - increment, increment = increment)
    elif QA == 'Try higher':
        threshold = uiThresholding(I, method = 'otsu', factorT = factorT + increment, increment = increment)
    elif QA == 'Yes':
        threshold = threshold
    return(threshold)
#

In [5]:
# M = np.array([A,B,C])
# np.sum(M, axis = 0)

In [6]:
# Test utility functions

# a = 'aa12-32-12AZF34_M2'
# datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", a)
# a[datePos.start():datePos.end()]
# findInfosInFileName(a, 'manipID')
# a = 'aaa_bbb'
# a.split('_')
# s = '21-04-27_M1_P1_C4_R40_disc20um_wFluo'
# findInfosInFileName(s, 'cellID')
# [(331.26256664936864, 258.949997248057), (403.98986101188945, 258.9821671859072)]

# XY1 = np.array([[1,2],[4,2],[7,2]])
# XY2 = np.array([[4.2,1.9],[7.3,2.4],[10.5,2.9]])
# M = compute_cost_matrix(XY1,XY2)
# row_ind, col_ind = linear_sum_assignment(M)
# for i in range(len(row_ind)):
#     e1 = XY1[row_ind[i]]
#     e2 = XY2[col_ind[i]]
#     print(str(e1) + ' match with ' + str(e2))
    
# XY1 = np.array([[1,2]])
# XY2 = np.array([[4.2,1.9],[7.3,2.4],[10.5,2.9]])
# M = compute_cost_matrix(XY1,XY2)
# row_ind, col_ind = linear_sum_assignment(M)
# for i in range(len(row_ind)):
#     e1 = XY1[row_ind[i]]
#     e2 = XY2[col_ind[i]]
#     print(str(e1) + ' match with ' + str(e2))
# print(M[0])


# XY1 = np.array([[369.12113844, 315.78709427],
#  [240.68758168, 337.45733496],
#  [307.94786575, 347.40585667],
#  [170.31661143, 350.32238733],
#  [107.47383373, 372.72760651],
#  [ 36.83548803, 376.5760606 ]])
# XY2 = np.array([[369.4600591,  316.04924013],
#  [240.50990651, 338.07534854],
#  [308.02942955, 347.83002833],
# #  [170.36161867, 350.45379052],
#  [107.29199259, 372.72822661],
#  [ 36.74465239, 376.59989911]])
# M = compute_cost_matrix(XY1,XY2)
# print(M)
# row_ind, col_ind = linear_sum_assignment(M)
# for i in range(len(row_ind)):
#     e1 = XY1[row_ind[i]]
#     e2 = XY2[col_ind[i]]
#     print(str(e1) + ' match with ' + str(e2))
# costs = [M[row_ind[iB],col_ind[iB]] for iB in range(len(row_ind))]
# print(costs)


# A = np.array([10,9,8,7,8,9,10,10,10,10,10,10,10])
# B = np.array([10,10,10,9,8,7,8,9,10,10,10,10,10])
# C = np.array([10,10,10,10,10,9,8,7,8,9,10,10,10])

# listD = np.array([A,B,C])
# listStatus = [1,2,3]
# Nup = 3
# NVox = 2

# assembleDists(listD, listStatus, Nup, NVox)

### Tracker Classes

In [7]:
class PincherTimeLapse:
    """
    This class is initialised for each new .tif file analysed.
    
    It requires the following inputs :
    > I : the timelapse analysed.
    > cellID : the id of the cell currently analysed.
    > manipDict : the line of the experimental data table that concerns the current experiment.
    > NB : the number of beads of interest that will be tracked.
    
    It contains:
    * data about the 3D image I (dimensions = time, height, width),
    * a list of Frame objects listFrames, 1 per frame in the timelapse,
    * a list of Trajectory objects listTrajectories, 1 per bead of interest (Boi) in the timelapse,
    * a dictionnary dictLog, saving the status of each frame (see below) 
                             and all of the user inputs (points and clicks) during the tracking,
    * a pandas DataFrame detectBeadsResult, that contains the raw output of the bead tracking,
    * metadata about the experiment (cellID, expType, loopStruct, loop_totalSize, loop_rampSize, 
                                        loop_excludedSize, nLoop, Nuplet, blackFramesPerLoop).
    
    When a PincherTimeLapse is initialised, most of these variables are initialised to zero values.
    In order to compute the different fields, the following methods should be called in this order:
    - ptl.checkIfBlackFrames() : detect if there are black images at the end of each loop in the time lapse and 
                                 classify them as not relevant by filling the appropriate fields.
    - ptl.saveFluoAside() : save the fluo images in an other folder and classify them as not relevant 
                            for the rest of the image analysis.
    - ptl.determineFramesStatus() : fill the status and status_2 column of the dictLog.
                                    in the status field: -1 means excluded ; 0 means ramp ; >0 means *position in* the n-uplet
                                    in the status_2 field: -1 means excluded ; 0 means ramp ; >0 means *number of* the n-uplet
    - ptl.computeThreshold() : 
    - ptl.makeFramesList() :
    - ptl.detectBeads() :
    - ptl.findBestStd() :
    - ptl.buildTrajectories() :
    """
    
    def __init__(self, I, cellID, manipDict, NB = 2):
        nS, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        self.I = I
        self.threshold = 0
        self.NB = NB
        self.nx = nx
        self.ny = ny
        self.nS = nS
        self.listFrames = []
        self.listTrajectories = []
        self.cellID = cellID
        self.expType = manipDict['experimentType']
        self.scale = manipDict['scale pixel per um']
        self.OptCorrFactor = manipDict['optical index correction']
        self.MagCorrFactor = manipDict['magnetic field correction']
        self.BeadsZDelta = manipDict['beads bright spot delta']
        self.BeadTypeStr = manipDict['bead type']

        self.beadTypes = [bT for bT in str(manipDict['bead type']).split('_')]
        self.beadDiameters = [int(bD) for bD in str(manipDict['bead diameter']).split('_')]
        self.dictBeadDiameters = {}
        for k in range(len(self.beadTypes)):
            self.dictBeadDiameters[self.beadTypes[k]] = self.beadDiameters[k]
        
        self.dictLog = {'Slice' : np.array([i+1 for i in range(nS)]),
                        'Status' : np.zeros(nS, dtype = int),  # in the status field: -1 means excluded ; 0 means ramp ; >0 means position in the n-uplet
                        'Status_2' : np.zeros(nS, dtype = int), # in the status_2 field: -1 means excluded ; 0 means ramp ; >0 means number of the n-uplet
#                         Deprecated fields:
#                         'Fluo' : np.zeros(nS, dtype = bool),
#                         'Black' : np.zeros(nS, dtype = bool),
                        'UI' : np.zeros(nS, dtype = bool),
                        'UILog' : np.array(['' for i in range(nS)], dtype = '<U8'),
                        'UIxy' : np.zeros((nS,NB,2), dtype = int)}
        
        self.detectBeadsResult = pd.DataFrame({'Area' : [], 
                                               'StdDev' : [], 
                                               'XM' : [], 
                                               'YM' : [], 
                                               'Slice' : []})
#         self.wFluo = bool(manipDict['with fluo images'])

        loopStruct = manipDict['loop structure'].split('_')
        self.loop_totalSize = int(loopStruct[0])
        if self.expType == 'compressions':
            self.loop_rampSize = int(loopStruct[1])
        else:
            self.loop_rampSize = 0
        
        if len(loopStruct) == 3: # This 3rd part of the 'loopStruct' field is the nb of frames at the end
        # of each loop which are not part of the main analysis and should be excluded. Typically fluo images.
            self.loop_excludedSize = int(loopStruct[2])
        else:
            self.loop_excludedSize = 0
        
        self.nLoop = int(np.round(nS/self.loop_totalSize))
        self.Nuplet = manipDict['normal field multi images']
        self.Zstep = manipDict['multi image Z step']
        self.blackFramesPerLoop = np.zeros(self.nLoop)
       
    def checkIfBlackFrames(self):
        for i in range(self.nLoop):
            j = ((i+1)*self.loop_totalSize) - 1
            checkSum = np.sum(self.I[j])
            while checkSum == 0:
#                 self.dictLog['Black'][j] = True
                self.dictLog['Status'][j] = -1
                self.dictLog['Status_2'][j] = -1
                self.blackFramesPerLoop[i] += 1
                j -= 1
                checkSum = np.sum(self.I[j])
              
    def saveFluoAside(self, fluoDirPath = ''):
        if self.loop_excludedSize == 1:
#             if not os.path.exists(fluoDirPath):
#                 os.makedirs(fluoDirPath)
            for i in range(self.nLoop):
                j = int(((i+1)*self.loop_totalSize) - 1 - self.blackFramesPerLoop[i])
#                 self.dictLog['Fluo'][j] = True
                self.dictLog['Status'][j] = -1
                self.dictLog['Status_2'][j] = -1
                
    def determineFramesStatus_R40(self):
        N0 = self.loop_totalSize
        Nramp0 = self.loop_rampSize
        Nexclu = self.loop_excludedSize
        nUp = self.Nuplet
        N = N0 - Nexclu
        Nct = N - Nramp0
        i_nUp = 1
        for i in range(self.nLoop):
            jstart = int(i*N0)
            if Nramp0 == 0:
                for j in range(N):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                    self.dictLog['Status_2'][j] = i_nUp + j//self.Nuplet
            else:
                Nramp = Nramp0-self.blackFramesPerLoop[i]
                for j in range(Nct//2):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                    self.dictLog['Status_2'][jstart + j] = i_nUp + j//self.Nuplet
                i_nUp = max(self.dictLog['Status_2']) + 1
                jstart += int(Nct//2 + Nramp)
                for j in range(Nct//2):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                    self.dictLog['Status_2'][jstart + j] = i_nUp + j//self.Nuplet
                i_nUp = max(self.dictLog['Status_2']) + 1
                
    def saveLog(self, display = 1, save = False, path = ''):
        dL = {}
        dL['Slice'], dL['Status'], dL['Status_2'] = \
            self.dictLog['Slice'], self.dictLog['Status'], self.dictLog['Status_2']
#         dL['Fluo'], dL['Black'] = \
#             self.dictLog['Fluo'], self.dictLog['Black']
        dL['UI'], dL['UILog'] = \
            self.dictLog['UI'], self.dictLog['UILog']
        for i in range(self.NB):
            dL['UIx'+str(i+1)] = self.dictLog['UIxy'][:,i,0]
            dL['UIy'+str(i+1)] = self.dictLog['UIxy'][:,i,1]
        dfLog = pd.DataFrame(dL)
        if display == 1:
            print('\n\n* Initialized Log Table:\n')
            print(dfLog)
        if display == 2:
            print('\n\n* Filled Log Table:\n')
            print(dfLog[dfLog['UI']])
        if save:
            dfLog.to_csv(path, sep='\t')
        
    def importLog(self, path):
        dfLog = pd.read_csv(path, sep='\t')
        dL = dfLog.to_dict()
        self.dictLog['Slice'], self.dictLog['Status'], self.dictLog['Status_2'] = \
            dfLog['Slice'].values, dfLog['Status'].values, dfLog['Status_2'].values
#         self.dictLog['Fluo'], self.dictLog['Black'] = \
#             dfLog['Fluo'].values, dfLog['Black'].values
        self.dictLog['UI'], self.dictLog['UILog'] = \
            dfLog['UI'].values, dfLog['UILog'].values
        for i in range(self.NB):
            xkey, ykey = 'UIx'+str(i+1), 'UIy'+str(i+1)
            self.dictLog['UIxy'][:,i,0] = dfLog[xkey].values
            self.dictLog['UIxy'][:,i,1] = dfLog[ykey].values
        
    def computeThreshold(self, method = 'otsu', factor = 0.35):
        # TBC
#         factorT = 0.8*(self.D == 4.5) + 0.6*(self.D == 2.7)
        factorT = 0.35
        threshold = factorT*filters.threshold_otsu(self.I)
        self.threshold = threshold
        
    def testThresholding(self):
        I_test = self.I[self.nS//2]
        I_thresh = I_test > self.threshold
        fig, ax = plt.subplots(1,1)
        ax.imshow(I_thresh, cmap = 'gray')
        fig.show()
        
    def uiThresholding(self, method = 'otsu', factorT = 0.45, increment = 0.05):
        threshold = factorT*filters.threshold_otsu(self.I)
        I_test = self.I[2]
        I_thresh = I_test > threshold
        fig, ax = plt.subplots(1,1)
        ax.imshow(I_thresh, cmap = 'gray')
        fig.show()
        QA = pyautogui.confirm(
                    text='Is the threshold satisfying?',
                    title='Confirm threshold', 
                    buttons=['Yes', 'Try lower', 'Try higher'])
        plt.close(fig)
        if QA == 'Try lower':
            self.uiThresholding(method = 'otsu', factorT = factorT - increment, increment = increment)
        elif QA == 'Try higher':
            self.uiThresholding(method = 'otsu', factorT = factorT + increment, increment = increment)
        elif QA == 'Yes':
            self.threshold = threshold
            
    def saveMetaData(self, path):
        dMD = {}
        dMD['cellID'] = self.cellID
        dMD['threshold'] = self.threshold
        dMD['NB'] = self.NB
        f = open(path, 'w')
        for k in dMD.keys():
            f.write(str(k))
            f.write('\t')
            f.write(str(dMD[k]))
            f.write('\n')
        f.close()
        
    def readMetaData(self, path, infoType):
        f = open(path, 'r')
        f_lines = f.readlines()
        dMD = {}
        for line in f_lines:
            splitLine = line.split('\t')
            try:
                dMD[splitLine[0]] = int(splitLine[1])
            except:
                try:
                    dMD[splitLine[0]] = float(splitLine[1])
                except:
                    dMD[splitLine[0]] = splitLine[1]
        f.close()
        return(dMD[infoType])
             
    def makeFramesList(self):
        for i in range(self.nS):
            status = self.dictLog['Status'][i]
            status_2 = self.dictLog['Status_2'][i]
            # The Nup field of a slice is = to self.Nuplet if the Status indicates that the frmae is part of a multi image n-uplet
            # Otherwise the image is "alone", like in a compression, and therefore Nup = 1
            Nup = self.Nuplet * (self.dictLog['Status'][i] > 0)  +  1 * (self.dictLog['Status'][i] <= 0)
            if self.dictLog['Status'][i] >= 0:
                self.listFrames.append(Frame(self.I[i], i, self.NB, self.threshold, Nup, status, status_2, self.scale))
    
    def detectBeads(self, resFileImported, display = False):
        for frame in self.listFrames: #[:3]:
            plot = 0
            # TEST #################################################################################
#             listPlot = [i for i in range(1685, 1700)]
#             if frame.iS in listPlot:
#                 plot = 1
            # TEST #################################################################################
            if not resFileImported:
                frame.detectBeads(plot)
                self.detectBeadsResult = pd.concat([self.detectBeadsResult, frame.resDf])
            else:
                resDf = self.detectBeadsResult.loc[self.detectBeadsResult['Slice'] == frame.iS+1]
                frame.resDf = resDf
                
            frame.makeListBeads()
            
        if not resFileImported:
            self.detectBeadsResult = self.detectBeadsResult.convert_dtypes()
            self.detectBeadsResult.reset_index(inplace=True)
            self.detectBeadsResult.drop(['index'], axis = 1, inplace=True)
        
        if display:
            print('\n\n* Detected Beads Result:\n')
            print(self.detectBeadsResult)

    def saveBeadsDetectResult(self, path=''):
        self.detectBeadsResult.to_csv(path, sep='\t')
    
    def importBeadsDetectResult(self, path=''):
        df = pd.read_csv(path, sep='\t')
        for c in df.columns:
            if 'Unnamed' in c:
                df.drop([c], axis = 1, inplace=True)
        self.detectBeadsResult = df
        
    def findBestStd_V0(self):
        """
        This ugly function is my best attempt to implement sth very simple in a robust way.
        In the 'status' field, -1 means excluded image, 0 means image that isn't part of a N-uplet of images, and k>0 means position in the N-uplet of images.
        For each image in the N-uplet, I want to reconsititute this N-uplet (meaning the list of Nuplet consecutive images numbered from 1 to Nuplet, minus the images eventually with no beads detected).
        Then for each N-uplet of images, i want to find the max standard deviation and report its position because it's for the max std that the X and Y detection is the most precise.
        An exemple: with these inputs:
        Nuplet = 3
        status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
            iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
           std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
        The function will return bestStd, a list of boolean with the same length.
        Where status = 0, bestStd = True (the image is not part of a N-uplet, thus it need to be analysed regardless of its std).
        Where satus > 0, the function will cut the lists in N-uplet of max size 3:
        status -> [1,2,3] ; [1,2] ; [ 3] ; [ 1, 2, 3] ; [ 1, 2]
            iS -> [0,1,2] ; [6,7] ; [11] ; [12,13,14] ; [15,16]
           std -> [1,5,9] ; [1,5] ; [ 8] ; [ 2, 6, 9] ; [ 2, 6]
             i -> [0,1,2] ; [6,7] ; [ 8] ; [ 9,10,11] ; [12,13]
        and then will find the best std in each of those fragment and put a True at that position (list 'i') in bestStd, so in this example, at i = 2, 7, 8, 11 ,13
        So the output is: bestStd = [False, False, True, True, True, True, False, True, True, False, False, True, False, True]
        """
        
        Nuplet = self.Nuplet
        status = self.listTrajectories[0].dict['Status']
        iS = self.listTrajectories[0].dict['iS']
        nT = self.listTrajectories[0].nT
        std = np.zeros(nT)
        for i in range(0,self.NB):
            std += np.array(self.listTrajectories[i].dict['StdDev'])
        
        bestStd = np.zeros(nT, dtype = bool)

        current_Nup_status = []
        current_Nup_iS = []
        current_Nup_std = []
        current_Nup_i = []
        for i in range(nT):
            if status[i] == 0:
                bestStd[i] = True
            else:
                if len(current_Nup_i) == 0:
                    current_Nup_status = [status[i]]
                    current_Nup_iS = [iS[i]]
                    current_Nup_std = [std[i]]
                    current_Nup_i = [i]
                else:
                    if status[i] > current_Nup_status[-1] and (iS[i]-current_Nup_iS[-1]) < Nuplet:
                        current_Nup_status.append(status[i])
                        current_Nup_iS.append(iS[i])
                        current_Nup_std.append(std[i])
                        current_Nup_i.append(i)
                    else:
    #                     print(current_Nup_status, current_Nup_iS, current_Nup_std, current_Nup_i)
                        i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
    #                     print(i_bestStdInCurrentNuplet)
                        i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
                        bestStd[i_bestStd] = True
                        # then:
                        current_Nup_status = [status[i]]
                        current_Nup_iS = [iS[i]]
                        current_Nup_std = [std[i]]
                        current_Nup_i = [i]

        # Need to do it one last time at the end
        i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
        i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
        bestStd[i_bestStd] = True

        return(bestStd)
    
    def findBestStd(self):
        """
        Simpler and better than findBestStd_V0 using the status_2 column of the dictLog.
        ---
        For each frame of the timelapse that belongs to a N-uplet, I want to reconsititute this N-uplet 
        (meaning the list of 'Nup' consecutive images numbered from 1 to Nup, 
        minus the images eventually with no beads detected).
        Then for each N-uplet of images, i want to find the max standard deviation 
        and report its position because it's for the max std that the X and Y detection is the most precise.
        ---
        This is very easy thanks to the 'status_2', because it contains a different number for each N-Uplet.
        """
        
        Nup = self.Nuplet
        nT = self.listTrajectories[0].nT
        status_2 = self.listTrajectories[0].dict['Status_2']
        std = np.zeros(nT)
        for i in range(self.NB):
            std += np.array(self.listTrajectories[i].dict['StdDev'])
        
        bestStd = np.zeros(nT, dtype = bool)
        i = 0
        while i < nT:
            if status_2[i] == 0:
                bestStd[i] = True
                i += 1
            elif status_2[i] > 0:
                s2 = status_2[i]
                L = [i]
                j = 0
                while i+j < nT-1 and status_2[i+j+1] == s2: # lazy evaluation of booleans
                    j += 1
                    L.append(i+j)
                #print(L)    
                loc_std = std[L]
                i_bestStd = i + int(np.argmax(loc_std))
                bestStd[i_bestStd] = True
                L = []
                i = i + j + 1

        return(bestStd)
        
    def buildTrajectories(self):
        """
        One of the main functions.
        """
        # NB: 'iF': index in a list of Frames ; 
        # 'iB': index in a list of Beads or a list of Trajectories ; 
        # 'iS': index of the slice in the image I
        # Boi refers to the 'Beads of interest', ie the beads that are being tracked
        
        init_iF = 0
        init_ok = False
        while not init_ok:
            init_iS = self.listFrames[init_iF].iS
            if not self.dictLog['UI'][init_iS]:
                self.listFrames[init_iF].show()
                mngr = plt.get_current_fig_manager()
                mngr.window.setGeometry(700,100,1000, 800)
                QA = pyautogui.confirm(
                    text='Can you point the beads of interest\nin the image ' + str(init_iS + 1) + '?',
                    title='Initialise tracker', 
                    buttons=['Yes', 'Next Frame', 'Quit'])
                if QA == 'Yes':
                    init_ok = True
                    ui = plt.ginput(self.NB, timeout=0)
                    uiXY = ui2array(ui)
                    fig = plt.gcf()
                    plt.close(fig)
                    self.dictLog['UI'][init_iS] = True
                    self.dictLog['UILog'][init_iS] = 'init_' + QA
                    self.dictLog['UIxy'][init_iS] = uiXY
                elif QA == 'Next Frame':
                    fig = plt.gcf()
                    plt.close(fig)
                    self.dictLog['UI'][init_iS] = True
                    self.dictLog['UILog'][init_iS] = 'init_' + QA
                    init_iF += 1
                else:
                    fig = plt.gcf()
                    plt.close(fig)
                    return('Bug')
            else:
                QA = self.dictLog['UILog'][init_iS]
                if QA == 'init_Yes':
                    init_ok = True
                    uiXY = self.dictLog['UIxy'][init_iS]
                elif QA == 'Next Frame':
                    init_iF += 1
                else:
                    aa
        
        init_BXY = self.listFrames[init_iF].beadsXYarray()
        M = compute_cost_matrix(uiXY,init_BXY)
        row_ind, col_ind = linear_sum_assignment(M) # row_ind -> clicks / col_ind -> listBeads
        # Sort the initial beads to have them ordered by growing x coord.
        sortM = np.array([[init_BXY[col_ind[i],0], col_ind[i]] for i in range(len(col_ind))])
        sortM = sortM[sortM[:, 0].argsort()]
        init_iBoi = sortM[:, 1].astype(int)
        init_BoiXY = np.array([init_BXY[col_ind[i]] for i in range(len(col_ind))])
        
        # Creation of the Trajectory objects
        for iB in range(self.NB):
            self.listTrajectories.append(Trajectory(self.I, self.listFrames, self.scale, self.Zstep))

            self.listTrajectories[iB].dict['Bead'].append(self.listFrames[init_iF].listBeads[init_iBoi[iB]])
            self.listTrajectories[iB].dict['iF'].append(init_iF)
            self.listTrajectories[iB].dict['iS'].append(self.listFrames[init_iF].iS)
            self.listTrajectories[iB].dict['iB'].append(init_iBoi[iB])
            self.listTrajectories[iB].dict['X'].append(init_BoiXY[iB][0])
            self.listTrajectories[iB].dict['Y'].append(init_BoiXY[iB][1])
            self.listTrajectories[iB].dict['StdDev'].append(self.listFrames[init_iF].beadsStdDevarray()[init_iBoi[iB]])
            self.listTrajectories[iB].dict['Status'].append(self.listFrames[init_iF].status)
            self.listTrajectories[iB].dict['Status_2'].append(self.listFrames[init_iF].status_2)
            self.listTrajectories[iB].dict['idxCompression'].append(1 * (self.listFrames[init_iF].status == 0))

        previous_iF = init_iF
        previous_iBoi = init_iBoi
        previous_BXY = init_BXY
        previous_BoiXY = init_BoiXY
        
        for iF in range(init_iF+1, len(self.listFrames)):
            validFrame = True                
            
            if len(self.listFrames[iF].listBeads) < self.NB:
                validFrame = False
            
            if validFrame:
                BXY = self.listFrames[iF].beadsXYarray()
                M = compute_cost_matrix(previous_BXY,BXY)
                row_ind, col_ind = linear_sum_assignment(M)
                costs = [M[row_ind[iB],col_ind[iB]] for iB in range(len(row_ind))]
                
                # Assess wether the algo should aks for user input
                askUI = False
                if (max(costs)**0.5) * (1/self.scale) > 1: # If the greatest distance travelled by any object is greater than 1um
                    askUI = True

                if not askUI: # Automatically assign the positions of the next beads
                    try:
                        iBoi = [col_ind[row_ind.tolist().index(iB)] for iB in previous_iBoi]
                        BoiXY = np.array([BXY[iB] for iB in iBoi])
                    except:
                        print('Error for ' + str(iF))
                        askUI = True

                if askUI: # Ask user input to asign the positions of the next beads
                    iS = self.listFrames[iF].iS
                    # Case 1: the UI has not been previously saved in the dictLog
                    # Then ask for UI and save it in the dictLog
                    if not self.dictLog['UI'][iS]:
                        # Display the image, plot beads positions and current trajectories & ask the question
                        self.listFrames[iF].show()
                        for iB in range(self.NB):
                            T = self.listTrajectories[iB]
                            ax = plt.gca()
                            T.plot(ax, iB)
                        mngr = plt.get_current_fig_manager()
                        mngr.window.setGeometry(700,100,1000, 800)
                        QA = pyautogui.confirm(
                            text='Can you point the beads of interest\nin the image ' + str(iS + 1) + '?',
                            title='', 
                            buttons=['No', 'Yes', 'Abort Mission!'])
                        
                        # According to the question's answer:
                        if QA == 'Yes':
                            ui = plt.ginput(self.NB, timeout=0)
                            uiXY = ui2array(ui)
                            fig = plt.gcf()
                            plt.close(fig)
                            self.dictLog['UI'][iS] = True
                            self.dictLog['UILog'][iS] = QA
                            self.dictLog['UIxy'][iS] = uiXY
                        elif QA == 'No':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
                            self.dictLog['UI'][iS] = True
                            self.dictLog['UILog'][iS] = QA
                        elif QA == 'Abort Mission!':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
                            return('Bug')
                    
                    # Case 2: the UI has been previously saved in the dictLog.
                    # Then just import the previous answer from the dictLog
                    else:
                        QA = self.dictLog['UILog'][iS]
                        if QA == 'Yes':
                            uiXY = self.dictLog['UIxy'][iS]
                            
                        elif QA == 'No':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
            
            # If there were more than NB objects, and then the QA wasn't 'No', then the frame is valid.
            if validFrame:
                M = compute_cost_matrix(uiXY,BXY)
                row_ind, col_ind = linear_sum_assignment(M)
                sortM = np.array([[BXY[col_ind[i],0], col_ind[i]] for i in range(len(col_ind))])
                sortM = sortM[sortM[:, 0].argsort()]
                iBoi = sortM[:, 1].astype(int)
                BoiXY = np.array([BXY[iB] for iB in iBoi])
                # idxCompression = 0 if not in a ramp, and = number of ramp else. Basically increase by 1 each time you have an interval between two ramps.
                idxCompression = (self.listFrames[iF].status == 0) * (max(self.listTrajectories[iB].dict['idxCompression']) + 1 * (self.listTrajectories[iB].dict['idxCompression'][-1] == 0))
                for iB in range(self.NB):
                    #
                    self.listTrajectories[iB].dict['Bead'].append(self.listFrames[iF].listBeads[iBoi[iB]])
                    self.listTrajectories[iB].dict['iF'].append(iF)
                    self.listTrajectories[iB].dict['iS'].append(self.listFrames[iF].iS)
                    self.listTrajectories[iB].dict['iB'].append(iBoi[iB])
                    self.listTrajectories[iB].dict['X'].append(BoiXY[iB][0])
                    self.listTrajectories[iB].dict['Y'].append(BoiXY[iB][1])
                    self.listTrajectories[iB].dict['StdDev'].append(self.listFrames[iF].beadsStdDevarray()[iBoi[iB]])
                    self.listTrajectories[iB].dict['Status'].append(self.listFrames[iF].status)
                    self.listTrajectories[iB].dict['Status_2'].append(self.listFrames[iF].status_2)
                    self.listTrajectories[iB].dict['idxCompression'].append(idxCompression)
                    
                previous_iF = iF
                previous_iBoi = iBoi
                previous_BXY = BXY
                previous_BoiXY = BoiXY
                
        for iB in range(self.NB):
            for k in self.listTrajectories[iB].dict.keys():
                self.listTrajectories[iB].dict[k] = np.array(self.listTrajectories[iB].dict[k])
                
        # Now we have a functional Trajectory object
        # it's time to refine it
        
        nT = len(self.listTrajectories[0].dict['Bead'])
        
        
        # (a) Add the pointer to the correct line of the _Field.txt file.
        # It's just exactly the iS already saved in the dict, except if there are black images at the end of loops.
        # In that case you have to skip the X lines corresponding to the end of the ramp part, X being the nb of black images at the end of the current loop
        # This is because when black images occurs, they do because of the high frame rate during ramp parts and thus replace these last ramp images.
        
        for iB in range(self.NB):
            self.listTrajectories[iB].dict['Zr'] = np.zeros(nT)
            self.listTrajectories[iB].nT = nT
            iField = []
            for i in range(nT):
                S = self.listTrajectories[iB].dict['iS'][i]
                iLoop = (S//self.loop_totalSize)
                offset = self.blackFramesPerLoop[iLoop]
                i_lim = iLoop*self.loop_totalSize + (self.loop_totalSize - ((self.loop_totalSize-self.loop_rampSize)//2) - (self.loop_excludedSize + offset))
                # i_lim is the first index after the end of the ramp
                addOffset = (S >= i_lim)
                SField = S + int(addOffset*offset)
                iField.append(SField)
            self.listTrajectories[iB].dict['iField'] = iField
            
        # (b) Find the image with the best std within each n-uplet
            
        bestStd = self.findBestStd()
        for i in range(self.NB):
            self.listTrajectories[i].dict['bestStd'] = bestStd
            
    def forceCompute(self, traj1, traj2, B0, D3, dx):
    
        # Magnetization functions
        def computeMag_M270(B):
            M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
            return(M)

        def computeMag_M450(B):
            M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
            return(M)

        dictMag = {'M270' : computeMag_M270, 'M450' : computeMag_M450}
        dictBeadTypes = {2.7 : 'M270', 4.5 : 'M450'}
        
        dictLogF = {'D3' : [], 'B0' : [], 'Btot_L' : [], 'Btot_R' : [], 'F00' : [], 'F0' : [], 'dF_L' : [], 'dF_R' : [], 'Ftot' : []}

        # Correction functions
        def Bind_neighbour(B, D_BoI, neighbourType):
            if neighbourType == '':
                return(0)

            else:
                D_neighbour = self.dictBeadDiameters[neighbourType]
                f_Mag = dictMag[neighbourType] # Appropriate magnetization function
                M_neighbour = f_Mag(B) # magnetization [A.m^-1]
                V_neighbour = (4/3)*np.pi*(D_neighbour/2)**3 # volume [nm^3]
                m_neighbour = M_neighbour*V_neighbour*1e-9 # magnetic moment [A.nm^2]

                D_tot = (D_BoI + D_neighbour)/2 # Center-to-center distance [nm]
                B_ind = 2e5*m_neighbour/(D_tot**3) # Inducted mag field [mT]
                return(B_ind)

        def deltaF_neighbour(m_BoI, B, D_BoI, D_BoI2, neighbourType):
            if neighbourType == '':
                return(0)

            else:
                D_neighbour = self.dictBeadDiameters[neighbourType]
                f_Mag = dictMag[neighbourType] # Appropriate magnetization function
                M_neighbour = f_Mag(B) # magnetization [A.m^-1]
                V_neighbour = (4/3)*np.pi*(D_neighbour/2)**3 # volume [nm^3]
                m_neighbour = M_neighbour*V_neighbour*1e-9 # magnetic moment [A.nm^2]

                D_tot = D_BoI/2 + D_BoI2 + D_neighbour/2
                deltaF = 3e5*m_BoI*m_neighbour/D_tot**4 # force [pN]
                return(deltaF)



        # Let's make sure traj1 is the left bead traj and traj2 the right one.
        avgX1 = np.mean(traj1.dict['X'])
        avgX2 = np.mean(traj2.dict['X'])
        if avgX1 < avgX2:
            traj_L, traj_R = traj1, traj2
        else:
            traj_L, traj_R = traj2, traj1

        # Get useful data
        BeadType_L, BeadType_R = dictBeadTypes[traj_L.D], dictBeadTypes[traj_R.D]
        Neighbours_BL = np.concatenate(([traj_L.dict['Neighbour_L']], [traj_L.dict['Neighbour_R']]), axis = 0)
        Neighbours_BR = np.concatenate(([traj_R.dict['Neighbour_L']], [traj_R.dict['Neighbour_R']]), axis = 0)
        D_L, D_R = self.dictBeadDiameters[BeadType_L], self.dictBeadDiameters[BeadType_R]

        nT = len(B0)
        D3nm = 1000*D3
        Dxnm = 1000*dx
        F = np.zeros(nT)

        # Maybe possible to process that faster on lists themselves
        for i in range(nT):
            # Appropriate magnetization functions
            f_Mag_L = dictMag[BeadType_L]
            f_Mag_R = dictMag[BeadType_R]

            # Btot = B0 + B inducted by potential left neighbour mag + B inducted by potential right neighbour mag
            Btot_L = B0[i] + Bind_neighbour(B0[i], D_L, Neighbours_BL[0,i]) + Bind_neighbour(B0[i], D_L, Neighbours_BL[1,i])
            Btot_R = B0[i] + Bind_neighbour(B0[i], D_R, Neighbours_BR[0,i]) + Bind_neighbour(B0[i], D_R, Neighbours_BR[1,i])

            # Magnetizations
            M_L = f_Mag_L(Btot_L)
            M_R = f_Mag_R(Btot_R)

            # Volumes
            V_L = (4/3)*np.pi*(D_L/2)**3 # volume [nm^3]
            V_R = (4/3)*np.pi*(D_R/2)**3 # volume [nm^3]

            # Magnetizations
            m_L = M_L * 1e-9 * V_L
            m_R = M_R * 1e-9 * V_R

            anglefactor = abs(3*(Dxnm[i]/D3nm[i])**2 - 1)

            # Forces
            F00 = 3e5*anglefactor * (f_Mag_L(B0[i])* 1e-9*V_L) * (f_Mag_R(B0[i])*1e-9*V_R) / (D3nm[i]**4)
            F0 = 3e5*anglefactor*m_L*m_R/D3nm[i]**4
            dF_L = deltaF_neighbour(m_L, B0[i], D_L, D_R, Neighbours_BR[1,i])
            dF_R = deltaF_neighbour(m_R, B0[i], D_R, D_L, Neighbours_BL[0,i])

            # Total force = force between beads involved in the pair (F0)
            #               + small force between B_L and B_R's potential right neighbour
            #               + small force between B_R and B_L's potential left neighbour
            F[i] = F0 + dF_L + dF_R
            
            dictLogF['D3'].append(D3nm[i]-(D_L+D_R)/2)
            dictLogF['B0'].append(B0[i])
            dictLogF['Btot_L'].append(Btot_L)
            dictLogF['Btot_R'].append(Btot_R)
            dictLogF['F00'].append(F00)
            dictLogF['F0'].append(F0)
            dictLogF['dF_L'].append(dF_L)
            dictLogF['dF_R'].append(dF_R)
            dictLogF['Ftot'].append(F[i])
        
        dfLogF = pd.DataFrame(dictLogF)
        
        return(F, dfLogF)
        
#

In [8]:
class Frame:
    def __init__(self, F, iS, NB, threshold, Nup, status, status_2, scale):
        ny, nx = F.shape[0], F.shape[1]
        self.F = F # Note : Frame.F points directly to the i-th frame of the image I ! To have 2 different versions one should use np.copy(F)
        self.threshold = threshold
        self.NBoi = NB
        self.NBdetected = 0
        self.nx = nx
        self.ny = ny
        self.iS = iS
        self.listBeads = []
        self.trajPoint = []
        self.Nuplet = Nup
        self.status = status
        self.status_2 = status_2
        self.scale = scale
        self.resDf = pd.DataFrame({'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})
 
    def __str__(self):
        text = 'a'
        return(text)
    
    def show(self, strech = True):
        fig, ax = plt.subplots(1,1)
#         fig_size = plt.gcf().get_size_inches()
#         fig.set_size_inches(2 * fig_size)
        if strech:
            pStart, pStop = np.percentile(self.F, (1, 99))
            ax.imshow(self.F, cmap = 'gray', vmin = pStart, vmax = pStop)
        else:
            ax.imshow(self.F, cmap = 'gray')
        if len(self.listBeads) > 0:
            for B in self.listBeads:
                ax.plot([B.x], [B.y], c='orange', marker='o')
        fig.show()
    
    def detectBeads(self, plot):
        F_bin = self.F > self.threshold
        F_lab, nObj = ndi.label(F_bin)
        props = measure.regionprops(F_lab)
        listValidLabels = []
        areas = np.zeros(nObj+1)
        
        if plot >= 1:
            A_plot = np.zeros(nObj+1)
            C_plot = np.zeros(nObj+1)
            Valid_Plot = np.zeros(nObj+1)
            Plot_Labels = []
        
        for k in range(1, nObj+1):
            try:
                bb = props[k-1].bbox
                Valid = not (min(bb) == 0 or bb[2] == self.ny or bb[3] == self.nx) # Remove objects touching the edges of the frame

    #             # OPTION 1 - Compute the metrics on the filled shape ; NB: takes a lot of time
    #             F_fh = ndi.binary_fill_holes((F_lab == k).astype(int))
    #             tmp_props = measure.regionprops(F_fh.astype(int))
    #             A = tmp_props[0].area
    #             P = tmp_props[0].perimeter
    #             Circ = (4 * np.pi * A) / (P * P)
    #             Valid = Valid and A >= 100 and Circ >= 0.75 # Area and circularity criterion

                # OPTION 2 - Lower the criterion in circularity - NB: less selective
                A = props[k-1].area
                P = props[k-1].perimeter
                Circ = (4 * np.pi * A) / (P * P)
                if A >= 100 and Circ >= 0.7:
                    pass
                elif A >= 300 and Circ >= 0.55:
                    F_fh = ndi.binary_fill_holes((F_lab == k).astype(int))
                    tmp_props = measure.regionprops(F_fh.astype(int))
                    A = tmp_props[0].area
                    P = tmp_props[0].perimeter
                    Circ = (4 * np.pi * A) / (P * P)
                
                
                Valid = (Valid and A >= 100 and Circ >= 0.65) # Area and circularity criterion
                
                if plot >= 1:
                    A_plot[k] = A
                    C_plot[k] = Circ
                    Valid_Plot[k] = Valid
                    Plot_Labels.append(k)
                

            except:
                Valid = False
                
            if Valid:
                listValidLabels.append(k)
                areas[k] = A
                
            
                
                
                
#         F_labValid, nObjValid = ndi.label(F_lab)    
#         fig, ax = plt.subplots(1,1)
#         ax.imshow(F_labValid)
#         fig.show()

        if plot >= 1:
            centroids_plot = np.array([ndi.center_of_mass(self.F, labels=F_lab, index=m) for m in Plot_Labels])
            color = np.where(Valid_Plot == True, 'green', 'red')
            fig, ax = plt.subplots(1,1)
            ax.imshow(F_lab)
            for m in Plot_Labels:
                ax.plot(centroids_plot[m-1,1], centroids_plot[m-1,0], marker = '+', color = color[m])
                S = 'A = {:.2f} ; C = {:.2f}'.format(A_plot[m], C_plot[m])
                ax.text(centroids_plot[m-1,1], centroids_plot[m-1,0], S, color = color[m])
            fig.show()
        
        centroids = np.array([ndi.center_of_mass(self.F, labels=F_lab, index=i) for i in listValidLabels])
        
        resDict = {}
        resDict['Area'] = np.array([areas[i] for i in listValidLabels]).astype(int)
        resDict['StdDev'] = np.array([ndi.standard_deviation(self.F, labels=F_lab, index=i) for i in listValidLabels])
        resDict['XM'] = centroids[:,1]
        resDict['YM'] = centroids[:,0]
        resDict['Slice'] = np.array([self.iS+1 for i in listValidLabels]).astype(int)
        self.resDf = pd.DataFrame(resDict)
        
#         print(self.resDict)

    def makeListBeads(self):
        self.NBdetected = self.resDf.shape[0]
        for i in range(self.NBdetected):
            d = {}
            for c in self.resDf.columns:
                d[c] = self.resDf[c].values[i]
            self.listBeads.append(Bead(d, self.F))
            
    def beadsXYarray(self):
        A = np.zeros((len(self.listBeads), 2))
        for i in range(len(self.listBeads)):
            b = self.listBeads[i]
            A[i,0], A[i,1] = b.x, b.y
        return(A)
    
    def beadsStdDevarray(self):
        A = np.zeros(len(self.listBeads))
        for i in range(len(self.listBeads)):
            b = self.listBeads[i]
            A[i] = b.std
        return(A)
    
    def detectDiameter(self, plot = 0):
        for i in range(len(self.listBeads)):
            B = self.listBeads[i]
            B.detectDiameter(self.nx, self.ny, self.scale, plot)
#             B = self.listBeads[i]
#             roughSize = np.floor(5*SCALE_100X)
#             roughSize += roughSize%2
#             x1_roughROI = max(int(np.floor(B.x) - roughSize*0.5) - 1, 0)
#             x2_roughROI = min(int(np.floor(B.x) + roughSize*0.5), self.nx)
#             y1_roughROI = max(int(np.floor(B.y) - roughSize*0.5) - 1, 0)
#             y2_roughROI = min(int(np.floor(B.y) + roughSize*0.5), self.ny)        
#             F_roughRoi = self.F[y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]            
#             if plot == 1:
#                 figh, axh = plt.subplots(1,1, figsize = (4,4))
#                 axh.hist(F_roughRoi.ravel(), bins=256, histtype='step', color='black')
#             counts, binEdges = np.histogram(F_roughRoi.ravel(), bins=256)
#             peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
#                                width=None, wlen=None, rel_height=0.5, plateau_size=None)       
#             peakThreshVal = 750    
#             if counts[peaks[0]] > peakThreshVal:
#                 B.D = 4.5      
#             else:
#                 B.D = 2.7   
#             if plot == 1:    
#                 print(B.D)
#                 B.show()
            pass
                
    def detectNeighbours(self, iB, beadType = 'detect'):
        BoI = self.listBeads[iB]
        listD = []
        for B in self.listBeads:
            if B.D not in listD:
                listD.append(B.D)
        Dmax = max(listD)*self.scale
        DBoI = BoI.D*self.scale
        x, y = BoI.x, BoI.y
#         print('Bead in (x,y) = (' + str(x) + ', ' + str(y) + ')')
        
        def makeRoI(x01, x02, y01, y02, nx, ny):
            x1, x2, y1, y2 = int(x01), int(x02), int(y01), int(y02)
            Lx = [x1, x2]
            Ly = [y1, y2]
            for x in Lx:
                if x < 0:
                    x = 0
                if x > nx:
                    x = nx
            for y in Ly:
                if y < 0:
                    y = 0
                if y > ny:
                    y = ny
            return([Lx, Ly])
        
        def areThereBeadsInRoI(listBeads, RoI):
            beadsFound = []
            Lx, Ly = RoI[0], RoI[1]
#             print(RoI)
            for B in listBeads:
#                 print(B.x, B.y)
                if B.x > Lx[0] and B.x < Lx[1] and B.y > Ly[0] and B.y < Ly[1]:
                    beadsFound.append(B)
            return(beadsFound)
        
        RoI_left = makeRoI(x-(0.5*DBoI+Dmax), x-(0.5*DBoI), y-0.55*Dmax, y+0.55*Dmax, self.nx, self.ny)
        RoI_right = makeRoI(x+(0.5*DBoI), x+(0.5*DBoI+Dmax), y-0.55*Dmax, y+0.55*Dmax, self.nx, self.ny)
        
        beadsFound_left = areThereBeadsInRoI(self.listBeads, RoI_left)
        beadsFound_right = areThereBeadsInRoI(self.listBeads, RoI_right)
        
        if len(beadsFound_left) >= 1:
            if beadType == 'detect':
                beadsFound_left[0].detectDiameter(self.nx, self.ny, self.scale, plot = 0)
                D = beadsFound_left[0].D
                if D == 4.5:
                    BoI.Neighbour_L = 'M450'
                elif D == 2.7:
                    BoI.Neighbour_L = 'M270'
            else:
                BoI.Neighbour_L = beadType
                
        if len(beadsFound_right) >= 1:
            if beadType == 'detect':
                beadsFound_right[0].detectDiameter(self.nx, self.ny, self.scale, plot = 0)
                D = beadsFound_right[0].D
                if D == 4.5:
                    BoI.Neighbour_R = 'M450'
                elif D == 2.7:
                    BoI.Neighbour_R = 'M270'
            else:
                BoI.Neighbour_R = beadType
        
        return(BoI.Neighbour_L, BoI.Neighbour_R)
    
#

In [9]:
class Bead:
    def __init__(self, d, F):
        self.x = d['XM']
        self.y = d['YM']
        self.D = 0
        self.area = d['Area']
        self.std = d['StdDev']
        self.iS = d['Slice']-1
        self.status = ''
        self.Neighbour_L = ''
        self.Neighbour_R = ''
        self.F = F
        
    def detectDiameter(self, nx, ny, scale, plot = 0):
        F = self.F
        roughSize = np.floor(5*scale)
        roughSize += roughSize%2
        x1_roughROI = max(int(np.floor(self.x) - roughSize*0.5) - 1, 0)
        x2_roughROI = min(int(np.floor(self.x) + roughSize*0.5), nx)
        y1_roughROI = max(int(np.floor(self.y) - roughSize*0.5) - 1, 0)
        y2_roughROI = min(int(np.floor(self.y) + roughSize*0.5), ny)

        F_roughRoi = F[y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]   

        if plot == 1:
            figh, axh = plt.subplots(1,1, figsize = (4,4))
            axh.hist(F_roughRoi.ravel(), bins=256, histtype='step', color='black')

        counts, binEdges = np.histogram(F_roughRoi.ravel(), bins=256)

        peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
                           width=None, wlen=None, rel_height=0.5, plateau_size=None)

        peakThreshVal = 750

        if counts[peaks[0]] > peakThreshVal:
            self.D = 4.5

        else:
            self.D = 2.7

        if plot == 1:    
            print(self.D)
            self.show()

    def show(self, strech = True):
        fig, ax = plt.subplots(1,1)
        if strech:
            pStart, pStop = np.percentile(self.F, (1, 99))
            ax.imshow(self.F, cmap = 'gray', vmin = pStart, vmax = pStop)
        else:
            ax.imshow(self.F, cmap = 'gray')
        ax.plot([self.x], [self.y], c='orange', marker='o')
        fig.show()

#

In [10]:
class Trajectory:
    def __init__(self, I, listFrames, scale, Zstep):
        nS, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        self.I = I
        self.listFrames = listFrames
        self.scale = scale
        self.nx = nx
        self.ny = ny
        self.nS = nS
        self.D = 0
        self.nT = 0
#         self.seriesBeads = []
#         self.pointerBeads = []
#         self.series_iS = []
#         self.seriesXY = []
#         self.current_iS = 0
        self.dict = {'X': [],'Y': [],'idxCompression': [],'StdDev': [], 
                     'Bead': [],'Status': [],'Status_2': [],'iF': [],'iS': [],'iB' : [],
                     'bestStd' : [], 'Zr' : [], 'Neighbour_L' : [], 'Neighbour_R' : []}
        # These columns will be added later : 'Neighbours', 'Z', 'bestStd' and 'iField'
        # iF is the index in the listFrames
        # iS is the index of the slice in the raw image
        
        self.deptho = []
        self.depthoPath = ''
        self.depthoStep = 20
        self.depthoZFocus = 200
        self.Zstep = Zstep # The step in microns between 2 consecutive frames in a multi-frame Nuplet
                
    def __str__(self):
        text = 'iS : ' + str(self.series_iS)
        text += '\n'
        text += 'XY : ' + str(self.seriesXY)
        return(text)
    
    def to_df(self):
        df = pd.DataFrame(self.dict)
        return(df)
    
    def computeZ(self, plot = 0):
        if len(self.deptho) == 0:
            return('Error, no depthograph associated with this trajectory')
        
        else:
            Ddz, Ddx = self.deptho.shape[0], self.deptho.shape[1]
            iF = self.dict['iF'][0]
            previousZ = -1
            while iF <= max(self.dict['iF']):
                plot = 0
############################ TEST ###################################
                plot = 0
                if iF <= 21:# or (iF < 190 and iF > 150):
                    plot = 1
############################ TEST ###################################
                if iF == 21:# or (iF < 190 and iF > 150):
                    plot = 1
    
                if iF not in self.dict['iF']:
                    iF += 1
                else:
                    F = self.listFrames[iF]
                    Nup = F.Nuplet
                    if Nup <= 1:
                        framesNuplet = [F]
                        iFNuplet = [iF]
                        iF += 1
                    elif Nup > 1:
                        framesNuplet = [F]
                        iFNuplet = [iF]
                        jF = 1
                        while iF+jF <= max(self.dict['iF']) and self.listFrames[iF+jF].status_2 == F.status_2:
                            if iF+jF in self.dict['iF']: # One of the images of the triplet may be invalid, 
                                # and we don't want to take it. With this test we won't
                                nextF = self.listFrames[iF+jF]
                                framesNuplet.append(nextF)
                                iFNuplet.append(iF+jF)
                            jF += 1
                            
                        iF += jF
                    
                    maxDz = 40
                    Z = self.findZ_Nuplet(framesNuplet, iFNuplet, Nup, previousZ, maxDz, plot)
                    previousZ = Z
                    # This Z_pix has no meaning in itself, it needs to be compared to the depthograph Z reference point,
                    # which is depthoZFocus. 
                    Zr = self.depthoZFocus - Z # If you want to find it back, Z = depthoZFocus - Zr
                    # This definition was chosen so that when Zr > 0, the plane of observation of the bead is HIGHER than the focus
                    # and accordingly when Zr < 0, the plane of observation of the bead is LOWER than the focus

                    mask = np.array([(iF in iFNuplet) for iF in self.dict['iF']])
                    self.dict['Zr'][mask] = Zr
                             
    def findZ_Nuplet(self, framesNuplet, iFNuplet, Nup, previousZ, maxDz = 40, plot = 0):
        Nframes = len(framesNuplet)
        listStatus_1 = [F.status for F in framesNuplet]
        listXY = [[self.dict['X'][np.where(self.dict['iF']==iF)][0], 
                   self.dict['Y'][np.where(self.dict['iF']==iF)][0]] for iF in iFNuplet]
        listiS = [self.dict['iS'][np.where(self.dict['iF']==iF)][0] for iF in iFNuplet]
        cleanSize = getDepthoCleanSize(self.D, self.scale)
        hdSize = self.deptho.shape[1]
        depthoDepth = self.deptho.shape[0]
        listProfiles = np.zeros((Nframes, hdSize))
        listROI = []

        for i in range(Nframes):
            xx = np.arange(0, 5)
            yy = np.arange(0, cleanSize)
            X, Y, iS = int(np.round(listXY[i][0])), int(np.round(listXY[i][1])), listiS[i] # > We could also try to recenter the image to keep a subpixel resolution here
            # line that is 5 pixels wide
            profileROI = framesNuplet[i].F[Y-cleanSize//2:Y+cleanSize//2+1, X-2:X+3]
            f = interpolate.interp2d(xx, yy, profileROI, kind='cubic')
            # Now use the obtained interpolation function and plot the result:
            xxnew = xx
            yynew = np.linspace(0, cleanSize, hdSize)
            profileROI_hd = f(xxnew, yynew)
            listROI.append(profileROI)

            listProfiles[i,:] = profileROI_hd[:,5//2] * (1/5)
            for j in range(1, 1 + 5//2):
                listProfiles[i,:] += profileROI_hd[:,5//2-j] * (1/5)
                listProfiles[i,:] += profileROI_hd[:,5//2+j] * (1/5)       

        listProfiles = listProfiles.astype(np.uint16)


        # now use listStatus_1, listProfiles, self.deptho + data about the jump between Nuplets ! (TBA) 
        # to compute the correlation function
        nVoxels = int(np.round(self.Zstep/self.depthoStep))
        listDistances = np.zeros((Nframes, depthoDepth))
        listZ = np.zeros(Nframes, dtype = int)
        for i in range(Nframes):
            listDistances[i] = squareDistance(self.deptho, listProfiles[i], normalize = True) # Utility functions
            listZ[i] = np.argmin(listDistances[i])
        # Translate the profiles that must be translated (status 1 & 3 if Nup = 3)
        # and don't move the others (status 2 if Nup = 3 or the 1 profile when Nup = 1)
        if Nup > 1:
            finalDists = assembleDists(listDistances, listStatus_1, Nup, nVoxels)
        elif Nup == 1:
            finalDists = listDistances
        sumFinalD = np.sum(finalDists, axis = 0)
        if previousZ == -1 or Nup > 1:
            Z = np.argmin(sumFinalD)
        elif Nup == 1 and previousZ != -1:
#             print(previousZ-maxDz, previousZ+maxDz)
            Z = previousZ-maxDz + np.argmin(sumFinalD[previousZ-maxDz:previousZ+maxDz])

        # PLOT
        if plot >= 1:
            fig, axes = plt.subplots(5, 3)
            im = framesNuplet[0].F
            X2, Y2 = listXY[0][0], listXY[0][1]
            
            pStart, pStop = np.percentile(im, (1, 99))
            axes[0,0].imshow(im, vmin = pStart, vmax = 1.5*pStop, cmap = 'gray')
            col_im = 'cyan'
            dx, dy = 10, 60
            axes[0,0].plot([X2], [Y2], marker = '+', color = 'red')
            axes[0,0].plot([X2-dx,X2-dx], [Y2-dy,Y2+dy], ls = '--', color = col_im)
            axes[0,0].plot([X2+dx,X2+dx], [Y2-dy,Y2+dy], ls = '--', color = col_im)
            axes[0,0].plot([X2-dx,X2+dx], [Y2-dy,Y2-dy], ls = '--', color = col_im)
            axes[0,0].plot([X2-dx,X2+dx], [Y2+dy,Y2+dy], ls = '--', color = col_im)
            
            axes[0,1].imshow(self.deptho)
            # TEST !!! # -> Works well !
            XL0, YL0 = axes[0,1].get_xlim(), axes[0,1].get_ylim()
            extent = (XL0[0], YL0[0]*(5/3), YL0[0], YL0[1])
            axes[0,1].imshow(self.deptho, extent = extent)
            # TEST !!! #

            pixLineHD = np.arange(0, hdSize, 1)
            zPos = np.arange(0, depthoDepth, 1)
            col = ['orange', 'gold', 'green']
            for i in range(Nframes):
#                 axes[0,1].plot([axes[0,1].get_xlim()[0], axes[0,1].get_xlim()[1]-1], [listZ[i], listZ[i]], ls = '--', c = col[i])
                axes[1,i].imshow(listROI[i])
                #
                axes[2,i].plot(pixLineHD, listProfiles[i])
                #
                axes[3,i].plot(zPos, listDistances[i])
                limy3 = axes[3,i].get_ylim()
                min_i = np.argmin(listDistances[i])
                axes[3,i].plot([min_i,min_i],limy3,ls = '--', c = col[i])
                #
                axes[4,i].plot(zPos, finalDists[i])
                limy4 = axes[4,i].get_ylim()
                min_i = np.argmin(finalDists[i])
                axes[4,i].plot([min_i,min_i],limy4,ls = '--', c = col[i])

#             axes[0,1].plot([axes[0,1].get_xlim()[0], axes[0,1].get_xlim()[1]-1], [Z,Z], ls = '--', c = 'red')
            axes[0,2].plot(zPos, sumFinalD)
            limy0 = axes[0,2].get_ylim()
            axes[0,2].plot([Z,Z],limy0,ls = '-', c = 'red')
            axes[0,2].plot([previousZ,previousZ],limy0,ls = '--', c = 'pink')
            axes[0,2].plot([previousZ-maxDz,previousZ-maxDz],limy0,ls = '--', c = 'cyan')
            axes[0,2].plot([previousZ+maxDz,previousZ+maxDz],limy0,ls = '--', c = 'cyan')

            fig.suptitle('Frames ' + str(iFNuplet) + ' ; Z = ' + str(Z))
            fig.show()

        return(Z)
    
    def keepBestStdOnly(self):
        dictBestStd = {}
        bestStd = self.dict['bestStd']
        nT = int(np.sum(bestStd))
        for k in self.dict.keys():
            A = np.array(self.dict[k])
            dictBestStd[k] = A[bestStd]
        self.dict = dictBestStd
        self.nT = nT
        
    def detectNeighbours(self, frequency, beadType = 'detect'):
        init = True
        listNeighbours = []
        for i in range(len(self.dict['iF'])):
            iS = self.dict['iS'][i]
            if frequency%iS == 0 or init:
                iF = self.dict['iF'][i]
                iB = self.dict['iB'][i]
                Frame = self.listFrames[iF]
                Neighbour_L, Neighbour_R = Frame.detectNeighbours(iB, beadType)
                listNeighbours.append([Neighbour_L, Neighbour_R])
                init = False
            else:
                self.dict['Bead'][i].Neighbour_L, self.dict['Bead'][i].Neighbour_R = Neighbour_L, Neighbour_R
                listNeighbours.append([Neighbour_L, Neighbour_R])
        arrayNeighbours = np.array(listNeighbours)
        self.dict['Neighbour_L'] = arrayNeighbours[:,0]
        self.dict['Neighbour_R'] = arrayNeighbours[:,1]
#         print(self.dict['Neighbour_L'], self.dict['Neighbour_R'])
            
    def plot(self, ax, i_color):
        colors = ['cyan', 'red', 'blue', 'orange']
        c = colors[i_color]
        ax.plot(self.dict['X'], self.dict['Y'], color=c, lw=0.5)
        
#

#### Test classes

In [11]:
def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

B0 = 6
D = 4503
R = D/2
V = (4/3)*np.pi*R**3
M = computeMag_M450(B0)
m = V*M*1e-9


Bind = (2e5*m)/(D**3)
Btot = B0 + 2*Bind
Mtot = computeMag_M450(Btot)
mtot = V*Mtot*1e-9

print('V = {:.3e}, M = {:.3e}, m = {:.3e}, Bind = {:.3e}, Btot = {:.3e}, Mtot = {:.3e}, mtot = {:.3e}'.format(V, M, m, Bind, Btot, Mtot, mtot))

V = 4.781e+10, M = 6.403e+03, m = 3.061e+05, Bind = 6.705e-01, Btot = 7.341e+00, Mtot = 7.363e+03, mtot = 3.520e+05


In [12]:
def mtot_2neighbours(B):
    def computeMag_M450(B):
        M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
        return(M)
    
    D = 4503
    R = D/2
    V = (4/3)*np.pi*R**3
    M = computeMag_M450(B)
    m = V*M*1e-9

    Bind = (2e5*m)/(D**3)
    Btot = B + 2*Bind
    Mtot = computeMag_M450(Btot)
    mtot = V*Mtot*1e-9
    
    return(m, mtot)
    
B = np.arange(0, 50, 1)
m, mtot = mtot_2neighbours(B)
fig, ax = plt.subplots(1,2)
ax[0].plot(B, m, 'b-', label = 'm')
ax[0].plot(B, mtot, 'r-', label = 'mtot')
ax[0].set_xlabel('B (mT)')
ax[0].set_ylabel('m (A.nm²)')
ax[0].legend()
ax[1].plot(B, mtot/m, 'k-', label = 'mtot/m')
ax[1].plot(B, mtot**2/m**2, 'k--', label = 'mtot/m')
ax[1].set_xlabel('B (mT)')
ax[1].set_ylabel('mtot/m')
ax[1].legend()
fig.show()
fig.suptitle('2 neighbours correction')

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


Text(0.5, 0.98, '2 neighbours correction')

In [34]:
def mtot_1and2neighbours_M450(B):
    def computeMag_M450(B):
        M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
        return(M)
    
    D = 4503
    R = D/2
    V = (4/3)*np.pi*R**3
    M = computeMag_M450(B)
    m = V*M*1e-9

    Bind = (2e5*m)/(D**3)
    Btot1 = B + Bind
    Btot2 = B + 2*Bind
    Mtot1 = computeMag_M450(Btot1)
    mtot1 = V*Mtot1*1e-9
    Mtot2 = computeMag_M450(Btot2)
    mtot2 = V*Mtot2*1e-9
    
    return(m, mtot1, mtot2)
    
B = np.arange(0, 50, 1)
m, mtot1, mtot2 = mtot_1and2neighbours_M450(B)

i = 0

fig, ax = plt.subplots(1,3)
ax[i].plot(B, m, 'b-', label = 'm - 0 neighbours')
ax[i].plot(B, mtot1, color = 'orange', ls = '-', label = 'm - 1 neighbours corr')
ax[i].plot(B, mtot2, color = 'red', ls = '-', label = 'm - 2 neighbours corr')
ax[i].set_xlabel('B (mT)')
ax[i].set_ylabel('m (A.nm²)')
ax[i].set_title('Corrections of m(B)')
ax[i].legend()
i += 1

# ax[i].plot(B, mtot1-m, color = 'orange', ls = '-', label = 'mtot1 - m')
# ax[i].plot(B, mtot2-m, color = 'red', ls = '-', label = 'mtot2 - m')
# ax[i].set_xlabel('B (mT)')
# ax[i].set_ylabel('m (A.nm²)')
# ax[i].set_title('Corrections of m(B)')
# ax[i].legend()
# i += 1

ax[i].plot(B, mtot1**2/m**2, color = 'gold', ls = '-', label = '0 neighbours corr')
ax[i].plot(B, mtot1*mtot2/m**2, color = 'orange', ls = '-', label = '1 neighbours corr')
ax[i].plot(B, mtot2**2/m**2, color = 'red', ls = '-', label = '2 neighbours corr')
ax[i].plot([5, 5], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'k', ls = '--', lw = 0.5)
# ax[i].plot([30, 30], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'cyan', ls = '-', lw = 0.5)
ax[i].set_xlabel('B (mT)')
ax[i].set_ylabel('Force increment ratio')
ax[i].set_title('Effects on the computed force')
ax[i].set_ylim([1.0, 1.7])
ax[i].legend()
i += 1

ax[i].plot(B, mtot1*mtot2/mtot1**2, color = 'orange', ls = '-', label = '1 neighbours corr / 0 neighbours corr')
ax[i].plot(B, mtot2**2/mtot1**2, color = 'red', ls = '-', label = '2 neighbours corr / 0 neighbours corr')
ax[i].plot([5, 5], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'k', ls = '--', lw = 0.5)
# ax[i].plot([30, 30], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'cyan', ls = '-', lw = 0.5)
ax[i].set_xlabel('B (mT)')
ax[i].set_ylabel('Ratio of ratio')
ax[i].set_title('Effects on the computed force')
ax[i].set_ylim([1.0, 1.4])
ax[i].legend()
i += 1

fig.show()
fig.suptitle('1 & 2 neighbours corrections\nto the magnetic moment - M450')

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in true_divide


Text(0.5, 0.98, '1 & 2 neighbours corrections\nto the magnetic moment - M450')

In [35]:
def mtot_1and2neighbours_M270(B):
    def computeMag_M270(B):
        M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
        return(M)
    
#     def computeMag_M270(B):
#         M = 0.74257*1.05*1600 * (153.4*B/158.1)
#         return(M)
    
    D = 2693
    R = D/2
    V = (4/3)*np.pi*R**3
    M = computeMag_M270(B)
    m = V*M*1e-9

    Bind = (2e5*m)/(D**3)
    Btot1 = B + Bind
    Btot2 = B + 2*Bind
    Mtot1 = computeMag_M270(Btot1)
    mtot1 = V*Mtot1*1e-9
    Mtot2 = computeMag_M270(Btot2)
    mtot2 = V*Mtot2*1e-9
    
    return(m, mtot1, mtot2)

B = np.arange(0, 50, 1)
m, mtot1, mtot2 = mtot_1and2neighbours_M270(B)

i = 0

fig, ax = plt.subplots(1,3)
ax[i].plot(B, m, 'b-', label = 'm - 0 neighbours')
ax[i].plot(B, mtot1, color = 'orange', ls = '-', label = 'm - 1 neighbours corr')
ax[i].plot(B, mtot2, color = 'red', ls = '-', label = 'm - 2 neighbours corr')
ax[i].set_xlabel('B (mT)')
ax[i].set_ylabel('m (A.nm²)')
ax[i].set_title('Corrections of m(B)')
ax[i].legend()
i += 1

# ax[i].plot(B, mtot1-m, color = 'orange', ls = '-', label = 'mtot1 - m')
# ax[i].plot(B, mtot2-m, color = 'red', ls = '-', label = 'mtot2 - m')
# ax[i].set_xlabel('B (mT)')
# ax[i].set_ylabel('m (A.nm²)')
# ax[i].set_title('Corrections of m(B)')
# ax[i].legend()
# i += 1

ax[i].plot(B, mtot1**2/m**2, color = 'gold', ls = '-', label = '0 neighbours corr')
ax[i].plot(B, mtot1*mtot2/m**2, color = 'orange', ls = '-', label = '1 neighbours corr')
ax[i].plot(B, mtot2**2/m**2, color = 'red', ls = '-', label = '2 neighbours corr')
ax[i].plot([5, 5], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'k', ls = '--', lw = 0.5)
# ax[i].plot([30, 30], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'cyan', ls = '-', lw = 0.5)
ax[i].set_xlabel('B (mT)')
ax[i].set_ylabel('Force increment ratio')
ax[i].set_title('Effects on the computed force')
ax[i].set_ylim([1.0, 1.7])
ax[i].legend()
i += 1

ax[i].plot(B, mtot1*mtot2/mtot1**2, color = 'orange', ls = '-', label = '1 neighbours corr / 0 neighbours corr')
ax[i].plot(B, mtot2**2/mtot1**2, color = 'red', ls = '-', label = '2 neighbours corr / 0 neighbours corr')
ax[i].plot([5, 5], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'k', ls = '--', lw = 0.5)
# ax[i].plot([30, 30], [ax[i].get_ylim()[0]*1.02, ax[i].get_ylim()[1]*0.98], color = 'cyan', ls = '-', lw = 0.5)
ax[i].set_xlabel('B (mT)')
ax[i].set_ylabel('Ratio of ratio')
ax[i].set_title('Effects on the computed force')
ax[i].set_ylim([1.0, 1.4])
ax[i].legend()
i += 1

fig.show()
fig.suptitle('1 & 2 neighbours corrections\nto the magnetic moment - M270')

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in true_divide
C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in true_divide


Text(0.5, 0.98, '1 & 2 neighbours corrections\nto the magnetic moment - M270')

In [20]:
# Test findBestStd

# def findBestStd(Nuplet, status, iS, std):
#     """
#     This ugly function is my best attempt to implement sth very simple in a robust way.
#     In the 'status' field, -1 means excluded image, 0 means image that isn't part of a N-uplet of images, and k>0 means position in the N-uplet of images.
#     For each image in the N-uplet, I want to reconsititute this N-uplet (meaning the list of Nuplet consecutive images numbered from 1 to Nuplet, minus the images eventually with no beads detected).
#     Then for each N-uplet of images, i want to find the max standard deviation and report its position because it's for the max std that the X and Y detection is the most precise.
#     An exemple: with these inputs:
#     Nuplet = 3
#     status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
#         iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
#        std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
#     The function will return bestStd, a list of boolean with the same length.
#     Where status = 0, bestStd = True (the image is not part of a N-uplet, thus it need to be analysed regardless of its std).
#     Where satus > 0, the function will cut the lists in N-uplet of max size 3:
#     status -> [1,2,3] ; [1,2] ; [ 3] ; [ 1, 2, 3] ; [ 1, 2]
#         iS -> [0,1,2] ; [6,7] ; [11] ; [12,13,14] ; [15,16]
#        std -> [1,5,9] ; [1,5] ; [ 8] ; [ 2, 6, 9] ; [ 2, 6]
#          i -> [0,1,2] ; [6,7] ; [ 8] ; [ 9,10,11] ; [12,13]
#     and then will find the best std in each of those fragment and put a True at that position (list 'i') in bestStd, so in this example, at i = 2, 7, 8, 11 ,13
#     So the output is: bestStd = [False, False, True, True, True, True, False, True, True, False, False, True, False, True]
#     """
    
#     Ntraj = len(iS)
#     bestStd = np.zeros(Ntraj, dtype = bool)
#     M = np.array([status, iS, std])
#     current_Nup_status = []
#     current_Nup_iS = []
#     current_Nup_std = []
#     current_Nup_i = []
#     for i in range(Ntraj):
#         if status[i] == 0:
#             bestStd[i] = True
#         else:
#             if len(current_Nup_i) == 0:
#                 current_Nup_status = [status[i]]
#                 current_Nup_iS = [iS[i]]
#                 current_Nup_std = [std[i]]
#                 current_Nup_i = [i]
#             else:
#                 if status[i] > current_Nup_status[-1] and (iS[i]-current_Nup_iS[-1]) < Nuplet:
#                     current_Nup_status.append(status[i])
#                     current_Nup_iS.append(iS[i])
#                     current_Nup_std.append(std[i])
#                     current_Nup_i.append(i)
#                 else:
# #                     print(current_Nup_status, current_Nup_iS, current_Nup_std, current_Nup_i)
#                     i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
# #                     print(i_bestStdInCurrentNuplet)
#                     i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
#                     bestStd[i_bestStd] = True
#                     # then:
#                     current_Nup_status = [status[i]]
#                     current_Nup_iS = [iS[i]]
#                     current_Nup_std = [std[i]]
#                     current_Nup_i = [i]
                    
#     # Need to do it one last time at the end
#     i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
#     i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
#     bestStd[i_bestStd] = True
    
#     return(bestStd)

# Nuplet = 3
# status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
# iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
# std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
# bs = findBestStd(Nuplet, status, iS, std)
# bs

In [24]:
plt.close('all')

In [25]:
# dictBestStd = {}
# dict1 = {'X': [1,1,1],'Y': [1,1,1],'idxCompression': [1,2,3],'StdDev': [10,20,30], 
#                      'Bead': [1,2,3],'Status': [1,2,3],'Status_2': [1,1,1],'iF': [1,2,3],'iS': [1,2,3],'iB' : [1,1,1],
#                      'bestStd' : [True,False,True], 'Zr' : [2,2,2]}
# bestStd = dict1['bestStd']
# for k in dict1.keys():
#     dict1[k] = np.array(dict1[k])
#     dictBestStd[k] = dict1[k][bestStd]
# dictBestStd

In [26]:
# A = np.array([1,2,3,4,5,6,7,8,9])
# B = A*10
# c = np.array([4,5,7])
# mask = np.array([(a in c) for a in A])
# B[mask] = 2
# fig, ax = plt.subplots(1,1)
# ax.plot(A,B)
# ylim = ax.get_ylim()
# ax.plot([4.5, 4.5], ylim, 'r--')

In [27]:
# A = np.ones((11,5))
# A[4,:] = np.array([1, 2, 6, 2, 1])
# A[5,:] = np.array([1, 4, 9, 4, 1])
# A[6,:] = np.array([1, 2, 8, 2, 1])
# A[7,:] = np.array([1, 5, 8, 4, 1])
# for i in range(11):
#     A[i,:] = (A[i,:] + A[i,::-1])/2
# v = np.array([1, 5, 8, 5, 1])
# v[:] = (v[:] + v[::-1])/2
# squareDistance(A,v)

In [28]:
# A = np.ones((11,5))
# A[4,:] = np.array([1, 2, 6, 2, 1])
# A[5,:] = np.array([1, 4, 9, 4, 1])
# A[6,:] = np.array([1, 2, 8, 2, 1])
# A[7,:] = np.array([1, 5, 8, 4, 1])
# for i in range(11):
#     A[i,:] = A[i,:] + A[i,::-1]
# v = np.array([[1, 5, 8, 4, 1]])
# v2 = v[:,::-1]
# v2 = v2/np.max(v2)
# print(v2)
# C = signal.correlate(A, v2, mode='valid', method='direct') # methodstr {‘auto’, ‘direct’, ‘fft’}
# n = C.shape[1]
# print(A)
# print(C)

### Deptho classes

Summary of deptho making

Initialise a MultiBeadsDeptho object
Use the build_listDepthos method on it
-> 

In [13]:
class MultiBeadsDeptho:
   
    def __init__(self, I, fileName, scale, Tmethod, Tfactor):
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.fileName = fileName
        self.scale = scale
        self.I_labeled = np.zeros([nz, ny, nx])
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.listDepthos = []
        self.ND = 0
        self.NDvalid = 0
        self.z_max = 0
#         self.threshold = 0.5*filters.threshold_otsu(I)
        self.threshold = uiThresholding(I, method = Tmethod, factorT = Tfactor, increment = 0.05)
        self.summaryTable = pd.DataFrame({})
        
    def build_listDepthos(self, size = 'detect', plot = 1):
        # Find a frame close to the focus for all beads
        maxVal = np.zeros(self.nz)
        for z in range(self.nz):
            maxVal[z] = np.max(self.I[z])
        z_max = np.argmax(maxVal)
        self.z_max = z_max
        F_max = self.I[z_max]
        
        # On the frame get the approximative position of the centers of all beads
        threshold_z_max = filters.threshold_otsu(F_max)
        F_max_bin = (F_max > threshold_z_max)
        F_labeled, ND = measure.label(F_max_bin, return_num = True)
        self.F_labeled, self.ND = F_labeled, ND
        
        # Summary plot, part 1/2
        if plot >= 1:
            pStart, pStop = np.percentile(F_max, (1, 99))
            figI, axI = plt.subplots(1,1, figsize = (15,5))
            axI.imshow(F_max, cmap = 'gray', vmin = pStart, vmax = pStop)
        
        # Building beadList
        iValid = 0
        for k in range(1,self.ND+1):
            # Initialisation of a BeadDeptho for each bright spot detected on F_max
            ym0, xm0 = ndi.center_of_mass(F_max, F_labeled, k)
            bD = BeadDeptho(self.I, xm0, ym0, self.z_max, self.threshold, self.fileName, self.scale)
            
            # Creation of the depthograph (if possible)
            bD.buildCleanROI(size, plot)
            
            # If the bead was not acceptable (for instance too close to the edge of the image)
            # then bD.validBead will be False
            if not bD.validBead:
                print('Not acceptable bead in x0, y0 = {:.2f}, {:.2f}'.format(bD.xm0, bD.ym0))
            # Else, we can proceed.
            else: 
                print('Job done for the bead in x0, y0 = {:.2f}, {:.2f}'.format(bD.xm0, bD.ym0))
                
                # Creation of the z profiles
                bD.buildDeptho(plot)
                
                # Each valid bead get assigned an index iValid
                bD.iValid = iValid
                iValid += 1
                
                # The beadDeptho is added to listDepthos
                self.listDepthos.append(bD)
                self.NDvalid += 1
                
                # Deprecated plot
#                 if plot >= 1:
#                     fig, ax = plt.subplots(1,1, figsize = (8, 5))
#                     dV = bD.depthosDict['deptho_v']
#                     pStart, pStop = np.percentile(dV, (1, 99))
#                     dV_rescaled = exposure.rescale_intensity(dV, in_range=(pStart, pStop))
#                     ax.imshow(dV_rescaled, cmap='gray')
#                     ax.set_ylim([bD.zLast, bD.zFirst])
            
            # Summary plot, part 2/2
            if plot >= 1:
                if bD.validBead:
                    axI.plot([bD.XYm[self.z_max, 0]], [bD.XYm[self.z_max, 1]], 'c+')
                    axI.text(bD.XYm[self.z_max, 0] + 5, bD.XYm[self.z_max, 1] - 5, str(bD.D), c = 'c')
                    bD.plotDeptho()
                else:
                    axI.plot([bD.xm0], [bD.ym0], 'r+')
            
            print('\n')  

In [14]:
# # Unused bit of code - Building summaryTable :
#         d = {'D' : [], 'x' : [], 'y' : []}
#         otherCols = [c for c in self.beadList[0].ZfocusDict.keys()]
#         for c in otherCols:
#             d[c] = []
#         for k in range(1,self.NDvalid+1):
#             bD = self.beadList[k-1]
#             d['D'].append(bD.D)
#             d['x'].append(bD.XYm[0,bD.z_max])
#             d['y'].append(bD.XYm[1,bD.z_max])
#             for c in otherCols:
#                 d[c].append(bD.ZfocusDict[c])        
#         self.summaryTable = pd.DataFrame(d)

In [15]:
class BeadDeptho:
    def __init__(self, I, xm0, ym0, z_max, threshold, fileName, scale):
        
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.scale = scale
        self.xm0 = xm0
        self.ym0 = ym0
        self.fileName = fileName
        
        self.D = 0
        self.threshold = threshold
        self.I_cleanROI = np.array([])
#         self.cleanROI = np.zeros((self.nz, 4), dtype = int)
        self.XYm = np.zeros((self.nz, 2))
        self.validBead = True
        self.iValid = -1
        
        self.z_max = z_max
        self.validSlice = np.zeros(nz, dtype = bool)
        self.zFirst = 0
        self.zLast = nz
        self.validDepth = nz
        
        self.valid_v = True
        self.valid_h = True
        self.depthosDict = {}
        self.profileDict = {}
        self.ZfocusDict = {}

        
    def buildCleanROI(self, size = 'detect', plot = 0):
        # Determine if the bead is to close to the edge on the max frame
        roughSize = np.floor(1.25*4.5*self.scale)
        x1, y1, x2, y2, validROI = getROI(roughSize, self.xm0, self.ym0, self.nx, self.ny)
        F_zmax = self.I[self.z_max, y1:y2, x1:x2]
        F_zmax_bin = (F_zmax > self.threshold)
        F_zmax_lab, nBzmax = measure.label(F_zmax_bin, return_num = True)
        
        validBead =  validROI and (nBzmax == 1)
        self.validBead = validBead
        
        # If it's the case we can proceed
        if validBead:
            # Detect or infer the size of the beads we are measuring
            if size == 'detect':
                counts, binEdges = np.histogram(self.I[self.z_max,y1:y2,x1:x2].ravel(), bins=256)
                peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
                                   width=None, wlen=None, rel_height=0.5, plateau_size=None)
                peakThreshVal = 1000
                if counts[peaks[0]] > peakThreshVal:
                    self.D = 4.5
                else:
                    self.D = 2.7
            
            elif size == 'M450':
                self.D = 4.5   
            elif size == 'M270':
                self.D = 2.7 
            else:
                print('invalid size input')
                return('invalid size input')
            
            # Initialise the list of centers of mass
            self.XYm[self.z_max] = [self.xm0, self.ym0]
            
            # Track this bead among all frames
            # a) from z_max to the last frame
            previous_BoiXY = np.array([self.xm0, self.ym0])
            for z in range(self.z_max+1, self.nz):
                previous_BoiXY = self.trackBeadDeptho(z, previous_BoiXY)
                if not self.validSlice[z]:
                    break
            
            # b) from z_max to the first frame
            previous_BoiXY = np.array([self.xm0, self.ym0])
            for z in range(self.z_max-1, -1, -1):
                previous_BoiXY = self.trackBeadDeptho(z, previous_BoiXY)
                if not self.validSlice[z]:
                    break
        
        if validBead:
            zFirst = np.argmax(self.validSlice)
            zLast = self.nz - np.argmax(self.validSlice[::-1])

            roughSize = int(np.floor(1.2*self.D*self.scale))
            roughSize += 1 + roughSize%2
            roughCenter = int((roughSize+1)//2)

            cleanSize = getDepthoCleanSize(self.D, self.scale)
            

            I_cleanROI = np.zeros([self.nz, cleanSize, cleanSize])

            try:
                for i in range(zFirst, zLast):
                    xm0, ym0 = self.XYm[i,0], self.XYm[i,1]
                    x1, y1, x2, y2, validBead = getROI(roughSize, xm0, ym0, self.nx, self.ny)
                    if not validBead:
                        if x1 < 0 or x2 > self.nx:
                            self.valid_h = False
                        if y1 < 0 or y2 > self.ny:
                            self.valid_v = False

                    xm1, ym1 = xm0-x1, ym0-y1
                    I_roughRoi = self.I[i,y1:y2,x1:x2]

                    translation = (xm1-roughCenter, ym1-roughCenter)

                    tform = transform.EuclideanTransform(rotation=0, \
                                                         translation = (xm1-roughCenter, ym1-roughCenter))

                    I_tmp = transform.warp(I_roughRoi, tform, order = 1, preserve_range = True)

                    I_cleanROI[i] = np.copy(I_tmp[roughCenter-cleanSize//2:roughCenter+cleanSize//2+1,\
                                                  roughCenter-cleanSize//2:roughCenter+cleanSize//2+1])

                if not self.valid_v and not self.valid_h:
                    self.validBead = False

                else:
                    self.zFirst = zFirst
                    self.zLast = zLast
                    self.validDepth = zLast-zFirst
                    self.I_cleanROI = I_cleanROI.astype(np.uint16)

                # VISUALISE
                if plot >= 2:
                    for i in range(zFirst, zLast, 50):
                        self.plotROI(i)

            except:
                print('Error for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))

          
    def trackBeadDeptho(self, z, previous_BoiXY):
        self.validSlice[z] = True
        
        roughSize = np.floor(1.25*self.D*self.scale)
        x1, y1, x2, y2, validROI = getROI(roughSize, previous_BoiXY[0], previous_BoiXY[1], self.nx, self.ny)
        if not validROI:
            self.validSlice[z] = False
#             self.validBead = False
            return(previous_BoiXY)

        else:
            F_z = self.I[z, y1:y2, x1:x2]
            previous_BoiXY_ROI = previous_BoiXY - np.array([x1, y1])                  
            F_z_bin = (F_z > self.threshold)
            F_z_bin_fh = ndi.binary_fill_holes(F_z_bin).astype(int)
            F_z_lab, nB_ROI = measure.label(F_z_bin_fh, return_num = True)

            if nB_ROI == 0:
                self.validSlice[z] = False
                return(previous_BoiXY)

            else: # (if nB_ROI > 0)
                props = measure.regionprops(F_z_lab)
                listValidLabels = []
                for k in range(1, nB_ROI+1):
                    try:
                        bb = props[k-1].bbox
                        validObj = not (min(bb) == 0 or bb[2] == self.ny or bb[3] == self.nx) # Remove objects touching the edges of the frame
                        A = props[k-1].area
                        P = props[k-1].perimeter
                        Circ = (4 * np.pi * A) / (P * P)
                        validObj = validObj and A >= 50 and Circ >= 0.75 # Area and circularity criterion
                    except:
                        validObj = False
                    if validObj:
                        listValidLabels.append(k)

                if len(listValidLabels) == 0:
                    self.validSlice[z] = False
                    return(previous_BoiXY)

                else:
                    centroidsYX_list = [ndi.center_of_mass(F_z, labels=F_z_lab, index=i) for i in listValidLabels]
                    centroidsXY = np.array([[centroidsYX_list[k][1], centroidsYX_list[k][0]] for k in range(len(centroidsYX_list))])
                    M = compute_cost_matrix(np.array([previous_BoiXY_ROI]), centroidsXY)
                    i_best = np.argmin(M[0])
                    cost_best = M[0][i_best]**0.5

                    if cost_best/self.scale > 0.5:
                        fig, ax = plt.subplots(1,1, figsize = (8,8))
                        pStart, pStop = np.percentile(F_z, (1, 99))
                        ax.imshow(F_z, cmap = 'gray', vmin = pStart, vmax = pStop)
                        ax.plot(centroidsXY[:,0], centroidsXY[:,1], c='orange', marker='o')
                        fig.show()
                        mngr = plt.get_current_fig_manager()
                        mngr.window.setGeometry(700,100,1000, 800)
                        QA = pyautogui.confirm(
                            text='Can you point the bead of interest\nin the image ' + str(z + 1) + '?',
                            title='', 
                            buttons=['No', 'Yes', 'Abort Mission!'])
                        # According to the question's answer:
                        if QA == 'Yes':
                            ui = plt.ginput(1, timeout=0)
                            uiXY = ui2array(ui)
                            fig = plt.gcf()
                            plt.close(fig)
                            M = compute_cost_matrix(uiXY,centroidsXY)
                            i_best = np.argmin(M[0])
                        elif QA == 'No':
                            fig = plt.gcf()
                            plt.close(fig)
                            self.validSlice[z] = False
                        elif QA == 'Abort Mission!':
                            fig = plt.gcf()
                            plt.close(fig)
                            return('Abort!')
                            
                    if self.validSlice[z]:
                        BoiXY = np.array(centroidsXY[i_best,:]) + np.array([x1, y1])
                        self.XYm[z] = BoiXY
                        # self.validSlice[z] = True (already the case)
                        return(BoiXY)
                    else:
                        return(previous_BoiXY)
    

    def buildDeptho(self, plot = 0):
        preferedDeptho = 'v'
        side_ROI = self.I_cleanROI.shape[1]
        mid_ROI = side_ROI//2
        nbPixToAvg = 3 # Have to be an odd number
        deptho_v = np.zeros([self.nz, side_ROI], dtype = np.float64)
        deptho_h = np.zeros([self.nz, side_ROI], dtype = np.float64)
        deptho_HD = np.zeros([self.nz, side_ROI*5], dtype = np.float64)
        
        if self.valid_v:
            for z in range(self.zFirst, self.zLast):
                templine = side_ROI
                deptho_v[z] = self.I_cleanROI[z,:,mid_ROI] * (1/nbPixToAvg)
                for i in range(1, 1 + nbPixToAvg//2):
                    deptho_v[z] += self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) 
                    deptho_v[z] += self.I_cleanROI[z,:,mid_ROI + i] * (1/nbPixToAvg)
            deptho_v = deptho_v.astype(np.uint16)
            self.depthosDict['deptho_v'] = deptho_v
        
        if self.valid_h:
            for z in range(self.zFirst, self.zLast):
                templine = side_ROI
                deptho_h[z] = self.I_cleanROI[z,mid_ROI,:] * (1/nbPixToAvg)
                for i in range(1, 1 + nbPixToAvg//2):
                    deptho_h[z] += self.I_cleanROI[z,mid_ROI - i,:] * (1/nbPixToAvg) 
                    deptho_h[z] += self.I_cleanROI[z,mid_ROI + i,:] * (1/nbPixToAvg)
            deptho_h = deptho_h.astype(np.uint16)
            self.depthosDict['deptho_h'] = deptho_h
        
        if preferedDeptho == 'v' and not self.valid_v:
            hdDeptho = 'h'
        elif preferedDeptho == 'h' and not self.valid_h:
            hdDeptho = 'v'
        else:
            hdDeptho = preferedDeptho
            
        if hdDeptho == 'v':
            for z in range(self.zFirst, self.zLast):
                x = np.arange(mid_ROI - 2, mid_ROI + 3)
                y = np.arange(0, side_ROI)
#                 xx, yy = np.meshgrid(x, y)
                vals = self.I_cleanROI[z, :, mid_ROI-2:mid_ROI+3]
                f = interpolate.interp2d(x, y, vals, kind='cubic')
                # Now use the obtained interpolation function and plot the result:

                xnew = x
                ynew = np.arange(0, side_ROI, 0.2)
                vals_new = f(xnew, ynew)
                deptho_HD[z] = vals_new[:,5//2] * (1/nbPixToAvg)
                for i in range(1, 1 + nbPixToAvg//2):
                    deptho_HD[z] += vals_new[:,5//2-i] * (1/nbPixToAvg)
                    deptho_HD[z] += vals_new[:,5//2+i] * (1/nbPixToAvg)              
#                 if z == self.z_max:
#                     figInterp, axesInterp = plt.subplots(1,2)
#                     axesInterp[0].imshow(vals)
#                     axesInterp[0].plot([5//2, 5//2], [0, vals.shape[0]], 'r--')
#                     axesInterp[1].imshow(vals_new)
#                     axesInterp[1].plot([5//2, 5//2], [0, vals_new.shape[0]], 'r--')
#                     figInterp.show()     
            deptho_HD = deptho_HD.astype(np.uint16)    
            self.depthosDict['deptho_HD'] = deptho_HD
                
        elif hdDeptho == 'h':
            for z in range(self.zFirst, self.zLast):
                x = np.arange(0, side_ROI)
                y = np.arange(mid_ROI - 2, mid_ROI + 3)
#                 xx, yy = np.meshgrid(x, y)
                vals = self.I_cleanROI[z, mid_ROI-2:mid_ROI+3, :]
                f = interpolate.interp2d(x, y, vals, kind='cubic')
                # Now use the obtained interpolation function and plot the result:

                xnew = np.arange(0, side_ROI, 0.2)
                ynew = y
                vals_new = f(xnew, ynew)
                deptho_HD[z] = vals_new[5//2,:] * (1/nbPixToAvg)
                for i in range(1, 1 + nbPixToAvg//2):
                    deptho_HD[z] += vals_new[5//2-i,:] * (1/nbPixToAvg)
                    deptho_HD[z] += vals_new[5//2+i,:] * (1/nbPixToAvg)
#                 if z == self.z_max:
#                     figInterp, axesInterp = plt.subplots(1,2)
#                     axesInterp[0].imshow(vals)
#                     axesInterp[0].plot([0, vals.shape[1]], [5//2, 5//2], 'r--')
#                     axesInterp[1].imshow(vals_new)
#                     axesInterp[1].plot([0, vals_new.shape[1]], [5//2, 5//2], 'r--')
#                     figInterp.show()
            deptho_HD = deptho_HD.astype(np.uint16)    
            self.depthosDict['deptho_HD'] = deptho_HD
        
            
        
        # 3D caracterisation
        I_binary = np.zeros([self.I_cleanROI.shape[0], self.I_cleanROI.shape[1], self.I_cleanROI.shape[2]])
        I_binary[self.zFirst:self.zLast] = (self.I_cleanROI[self.zFirst:self.zLast] > self.threshold)
        Zm3D, Ym3D, Xm3D = ndi.center_of_mass(self.I_cleanROI, labels=I_binary, index=1)
        self.ZfocusDict['Zm3D'] = Zm3D
        
        # Raw profiles
        mid_ROI_HD = deptho_HD.shape[1]//2
        Z = np.array([z for z in range(self.I_cleanROI.shape[0])])
        intensity_tot = np.array([np.sum(self.I_cleanROI[z][I_binary[z].astype(bool)])/(1+np.sum(I_binary[z])) for z in range(self.I_cleanROI.shape[0])]).astype(np.float64)
        intensity_v = np.array([np.sum(deptho_v[z,:])/side_ROI for z in range(deptho_v.shape[0])]).astype(np.float64)
        intensity_h = np.array([np.sum(deptho_h[z,:])/side_ROI for z in range(deptho_h.shape[0])]).astype(np.float64)
        intensity_HD = np.array([np.sum(deptho_HD[z,mid_ROI_HD-5:mid_ROI_HD+6])/11 for z in range(deptho_HD.shape[0])]).astype(np.float64)
#         
        Zm_v, Zm_h, Zm_tot = np.argmax(intensity_v), np.argmax(intensity_h), np.argmax(intensity_tot)
        Zm_HD = np.argmax(intensity_HD)
        
        self.profileDict['intensity_v'] = intensity_v
        self.profileDict['intensity_h'] = intensity_h
        self.profileDict['intensity_HD'] = intensity_HD
        self.profileDict['intensity_tot'] = intensity_tot
        self.ZfocusDict['Zm_v'] = Zm_v
        self.ZfocusDict['Zm_h'] = Zm_h
        self.ZfocusDict['Zm_HD'] = Zm_HD
        self.ZfocusDict['Zm_tot'] = Zm_tot


        # Smoothed profiles
        Z_hd = np.arange(0, self.I_cleanROI.shape[0], 0.2)
        intensity_v_hd = np.interp(Z_hd, Z, intensity_v)
        intensity_h_hd = np.interp(Z_hd, Z, intensity_h)
        intensity_HD_hd = np.interp(Z_hd, Z, intensity_HD)
        intensity_tot_hd = np.interp(Z_hd, Z, intensity_tot)
        
        intensity_v_smooth = savgol_filter(intensity_v_hd, 101, 5)
        intensity_h_smooth = savgol_filter(intensity_h_hd, 101, 5)
        intensity_HD_smooth = savgol_filter(intensity_HD_hd, 101, 5)
        intensity_tot_smooth = savgol_filter(intensity_tot_hd, 101, 5)
        
        Zm_v_hd, Zm_h_hd, Zm_tot_hd = Z_hd[np.argmax(intensity_v_smooth)], Z_hd[np.argmax(intensity_h_smooth)], Z_hd[np.argmax(intensity_tot_smooth)]
        Zm_HD_hd = Z_hd[np.argmax(intensity_HD_smooth)]
        
        self.profileDict['intensity_v_smooth'] = intensity_v_smooth 
        self.profileDict['intensity_h_smooth'] = intensity_h_smooth
        self.profileDict['intensity_HD_smooth'] = intensity_HD_smooth
        self.profileDict['intensity_tot_smooth'] = intensity_tot_smooth
        self.ZfocusDict['Zm_v_hd'] = Zm_v_hd 
        self.ZfocusDict['Zm_h_hd'] = Zm_h_hd
        self.ZfocusDict['Zm_HD_hd'] = Zm_HD_hd
        self.ZfocusDict['Zm_tot_hd'] = Zm_tot_hd
        
        # VISUALISE
        if plot >= 2:
            self.plotProfiles()            
            
            
    def saveBeadDeptho(self, path, manipID, bestDetphoType = 'HD', bestFocusType = 'HD_hd', step = 20):
        supDataDir = manipID + '_supData'
        supDataDirPath = os.path.join(path, supDataDir)
        if not os.path.exists(supDataDirPath):
            os.makedirs(supDataDirPath)
        
        cleanROIName = manipID + '_cleanROI.tif'
        cleanROIPath = os.path.join(path, cleanROIName)
        io.imsave(cleanROIPath, self.I_cleanROI)
        
        profilesRaw_keys = ['intensity_v', 'intensity_h', 'intensity_HD', 'intensity_tot']
        profileDictRaw = {k: self.profileDict[k] for k in profilesRaw_keys}
        profileDictRaw_df = pd.DataFrame(profileDictRaw)
        profileDictRaw_df.to_csv(os.path.join(supDataDirPath, 'profiles_raw.csv'))
        
        profilesSmooth_keys = ['intensity_v_smooth', 'intensity_h_smooth', 'intensity_HD_smooth', 'intensity_tot_smooth']
        profileDictSmooth = {k: self.profileDict[k] for k in profilesSmooth_keys}
        profileDictSmooth_df = pd.DataFrame(profileDictSmooth)
        profileDictSmooth_df.to_csv(os.path.join(supDataDirPath, 'profiles_smooth.csv'))
        
        ZfocusDict_df = pd.DataFrame(self.ZfocusDict, index = [1])
        ZfocusDict_df.to_csv(os.path.join(supDataDirPath, 'Zfoci.csv'))
        
        bestFocus = self.ZfocusDict['Zm_' + bestFocusType]
        metadataPath = os.path.join(path, manipID + '_Metadata.csv')
        with open(metadataPath, 'w') as f:
            f.write('step;bestFocus')
#             for k in self.ZfocusDict.keys():
#                 f.write(';')
#                 f.write(k)
            f.write('\n')
            f.write(str(step) + ';' + str(bestFocus))
#             for k in self.ZfocusDict.keys():
#                 f.write(';')
#                 f.write(str(self.ZfocusDict[k]))
                
        depthoPath = os.path.join(path, manipID + '_deptho.tif')
        bestDeptho = self.depthosDict['deptho_' + bestDetphoType]
        io.imsave(depthoPath, bestDeptho)
        
        
        
# Plot functions
        
    def plotXYm(self):
        fig, ax = plt.subplots(1,1)
        pStart, pStop = np.percentile(self.I[self.z_max], (1, 99))
        ax.imshow(self.I[self.z_max], cmap = 'gray', vmin = pStart, vmax = pStop)
        ax.plot(self.XYm[self.validSlice,0],self.XYm[self.validSlice,1],'r-')
        fig.show()
        
    def plotROI(self, i = 'auto'):
        if i == 'auto':
            i = self.z_max
        
        fig, ax = plt.subplots(1,3, figsize = (16,4))
        
        xm, ym = np.mean(self.XYm[self.validSlice,0]),  np.mean(self.XYm[self.validSlice,1])
        ROIsize_x = self.D*1.25*self.scale + (max(self.XYm[self.validSlice,0])-min(self.XYm[self.validSlice,0]))
        ROIsize_y = self.D*1.25*self.scale + (max(self.XYm[self.validSlice,1])-min(self.XYm[self.validSlice,1]))
        x1_ROI, y1_ROI, x2_ROI, y2_ROI = int(xm - ROIsize_x//2), int(ym - ROIsize_y//2), int(xm + ROIsize_x//2), int(ym + ROIsize_y//2)        

        pStart, pStop = np.percentile(self.I[i], (1, 99))
        ax[0].imshow(self.I[i], cmap = 'gray', vmin = pStart, vmax = pStop)
        ax[0].plot([x1_ROI,x1_ROI], [y1_ROI,y2_ROI], 'c--')
        ax[0].plot([x1_ROI,x2_ROI], [y2_ROI,y2_ROI], 'c--')
        ax[0].plot([x2_ROI,x2_ROI], [y1_ROI,y2_ROI], 'c--')
        ax[0].plot([x1_ROI,x2_ROI], [y1_ROI,y1_ROI], 'c--')

        I_ROI = self.I[i,y1_ROI:y2_ROI,x1_ROI:x2_ROI] 
        pStart, pStop = np.percentile(I_ROI, (1, 99))
        ax[1].imshow(I_ROI, cmap = 'gray', vmin = pStart, vmax = pStop)
        ax[1].plot(self.XYm[self.validSlice,0]-x1_ROI, self.XYm[self.validSlice,1]-y1_ROI, 'r-', lw=0.75)
        ax[1].plot(self.XYm[i,0]-x1_ROI, self.XYm[i,1]-y1_ROI, 'b+', lw=0.75)
        
        pStart, pStop = np.percentile(self.I_cleanROI[i], (1, 99))
        mid = self.I_cleanROI[i].shape[0]//2
        I_cleanROI_binary = (self.I_cleanROI[i] > self.threshold)
        y, x = ndi.center_of_mass(self.I_cleanROI[i], labels=I_cleanROI_binary, index=1)
        ax[2].imshow(self.I_cleanROI[i], cmap = 'gray', vmin = pStart, vmax = pStop)
        ax[2].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
        ax[2].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
        ax[2].plot([x],[y], 'b+')
        fig.show()

    def plotProfiles(self):
        Z = np.array([z for z in range(self.I_cleanROI.shape[0])])
        Z_hd = np.arange(0, self.I_cleanROI.shape[0], 0.2)
        intensity_v = self.profileDict['intensity_v']
        intensity_h = self.profileDict['intensity_h']
        intensity_HD = self.profileDict['intensity_HD']
        intensity_tot = self.profileDict['intensity_tot']
        Zm_v = self.ZfocusDict['Zm_v']
        Zm_h = self.ZfocusDict['Zm_h']
        Zm_HD = self.ZfocusDict['Zm_HD']
        Zm_tot = self.ZfocusDict['Zm_tot']
        intensity_v_smooth = self.profileDict['intensity_v_smooth']
        intensity_h_smooth = self.profileDict['intensity_h_smooth']
        intensity_HD_smooth = self.profileDict['intensity_HD_smooth']
        intensity_tot_smooth = self.profileDict['intensity_tot_smooth']
        Zm_v_hd = self.ZfocusDict['Zm_v_hd']
        Zm_h_hd = self.ZfocusDict['Zm_h_hd']
        Zm_HD_hd = self.ZfocusDict['Zm_HD_hd']
        Zm_tot_hd = self.ZfocusDict['Zm_tot_hd']
        
#             fig, ax = plt.subplots(1,3, figsize = (12, 4))
#             ax[0].plot(Z, intensity_v)
#             ax[1].plot(Z, intensity_h)
#             ax[2].plot(Z, (intensity_tot))
#             ax[0].plot([Zm_v, Zm_v], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v = {:.2f}'.format(Zm_v))
#             ax[1].plot([Zm_h, Zm_h], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h = {:.2f}'.format(Zm_h))
#             ax[2].plot([Zm_tot, Zm_tot], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot = {:.2f}'.format(Zm_tot))
#             ax[0].legend(loc = 'lower right')
#             ax[1].legend(loc = 'lower right')
#             ax[2].legend(loc = 'lower right')
        
        fig, ax = plt.subplots(1,4, figsize = (16, 4))
        ax[0].plot(Z, intensity_v, 'b-')
        ax[1].plot(Z, intensity_h, 'b-')
        ax[2].plot(Z, intensity_HD, 'b-')
        ax[3].plot(Z, (intensity_tot), 'b-')
        ax[0].plot(Z_hd, intensity_v_smooth, 'k--')
        ax[1].plot(Z_hd, intensity_h_smooth, 'k--')
        ax[2].plot(Z_hd, intensity_HD_smooth, 'k--')
        ax[3].plot(Z_hd, intensity_tot_smooth, 'k--')
        ax[0].plot([Zm_v_hd, Zm_v_hd], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v_hd = {:.2f}'.format(Zm_v_hd))
        ax[1].plot([Zm_h_hd, Zm_h_hd], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h_hd = {:.2f}'.format(Zm_h_hd))
        ax[2].plot([Zm_HD_hd, Zm_HD_hd], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_HD_hd = {:.2f}'.format(Zm_HD_hd))
        ax[3].plot([Zm_tot_hd, Zm_tot_hd], [0, ax[3].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot_hd = {:.2f}'.format(Zm_tot_hd))
        ax[0].legend(loc = 'lower right')
        ax[1].legend(loc = 'lower right')
        ax[2].legend(loc = 'lower right')
        ax[3].legend(loc = 'lower right')

        #         print('Zm_v = {:.2f}, Zm_h = {:.2f}, Zm_tot = {:.2f}'\
        #               .format(Zm_v, Zm_h, Zm_tot))
        #         print('Zm_v_hd = {:.2f}, Zm_h_hd = {:.2f}, Zm_tot_hd = {:.2f}'\
        #               .format(Zm_v_hd, Zm_h_hd, Zm_tot_hd))
        
        fig.show()
        
    def plotDeptho(self, d = 'HD'):
        fig, ax = plt.subplots(1,1, figsize = (4, 6))
        D = self.depthosDict['deptho_' + d]
        z_focus = self.ZfocusDict['Zm_' + d + '_hd']
        ny, nx = D.shape[0], D.shape[1]
        pStart, pStop = np.percentile(D, (1, 99))
        pStop = pStop + 0.3 * (pStop-pStart)
        ax.imshow(D, cmap='plasma', vmin = pStart, vmax = pStop)
        ax.plot([0, nx], [self.zFirst, self.zFirst], 'r--')
        ax.text(nx//2, self.zFirst - 10, str(self.zFirst), c = 'r')
        ax.plot([0, nx], [self.zLast, self.zLast], 'r--')
        ax.text(nx//2, self.zLast - 10, str(self.zLast), c = 'r')
        ax.plot([nx//2], [z_focus], 'c+')
        ax.text(nx//2, z_focus - 10, str(z_focus), c = 'c')
        fig.suptitle('File ' + self.fileName + ' - Bead ' + str(self.iValid))
        fig.show()

In [16]:
plt.close('all')

In [17]:
def depthoMaker(dirPath, savePath, specif, scale, beadSize = 'M450', step = 20, d = 'HD', Tmethod = 'otsu', Tfactor = 1.5):
    listDepthosFileNames = [f for f in os.listdir(dirPath) if (os.path.isfile(os.path.join(dirPath, f)) 
                                                               and f.endswith(".tif"))]
    listMBDepthos = []
    dictBD = {}

    for f in listDepthosFileNames:
        filePath = os.path.join(dirPath, f)
        print(filePath)
        I = io.imread(filePath)
        mbd = MultiBeadsDeptho(I, f[:-4], scale, Tmethod = Tmethod, Tfactor = Tfactor)
        mbd.build_listDepthos(size = beadSize)
        listMBDepthos.append(mbd)
        i = 1
        for bD in mbd.listDepthos:
#             bD_manipID = findInfosInFileName(bD.fileName, 'manipID')
            subFileSavePath = os.path.join(savePath, 'Intermediate_Py', specif + '_' + str(int(bD.D*100)) + '_step' + str(step))
            bD.saveBeadDeptho(subFileSavePath, specif +  '_' + str(i), bestDetphoType = 'HD', bestFocusType = 'HD_hd', step = 20)
            i += 1
            if bD.D not in dictBD.keys():
                dictBD[bD.D] = [bD]
            else:
                dictBD[bD.D].append(bD)

    for size in dictBD.keys():
        listBD = dictBD[size]
        nBD = len(listBD)
        maxAboveZm, maxBelowZm = 0, 0
        for bD in listBD:
            Zm = int(np.round(bD.ZfocusDict['Zm_' + d + '_hd']))
            if Zm - bD.zFirst > maxAboveZm:
                maxAboveZm = Zm - bD.zFirst
            if bD.zLast - Zm > maxBelowZm:
                maxBelowZm = bD.zLast - Zm
        maxAboveZm, maxBelowZm = int(maxAboveZm), int(maxBelowZm)
        Zfocus = maxAboveZm
        depthoWidth = listBD[0].depthosDict['deptho_' + d].shape[1]
        depthoHeight = maxAboveZm + maxBelowZm
        finalDeptho = np.zeros([depthoHeight, depthoWidth], dtype = np.float64)

        for z in range(1, maxAboveZm+1):
            count = 0
            for bD in listBD:
                Zm = int(np.round(bD.ZfocusDict['Zm_' + d + '_hd']))
                currentDeptho = bD.depthosDict['deptho_' + d]
                if Zm-z >= 0 and np.sum(currentDeptho[Zm-z,:] != 0):
                    count += 1
            for bD in listBD:
                Zm = int(np.round(bD.ZfocusDict['Zm_' + d + '_hd']))
                currentDeptho = bD.depthosDict['deptho_' + d]
                if Zm-z >= 0 and np.sum(currentDeptho[Zm-z,:] != 0):
                    finalDeptho[Zfocus-z,:] += currentDeptho[Zm-z,:]/count
                    
        for z in range(0, maxBelowZm):
            count = 0
            for bD in listBD:
                Zm = int(np.round(bD.ZfocusDict['Zm_' + d + '_hd']))
                currentDeptho = bD.depthosDict['deptho_' + d]
                if Zm+z >= 0 and np.sum(currentDeptho[Zm+z,:] != 0):
                    count += 1
            for bD in listBD:
                Zm = int(np.round(bD.ZfocusDict['Zm_' + d + '_hd']))
                currentDeptho = bD.depthosDict['deptho_' + d]
                if Zm+z >= 0 and np.sum(currentDeptho[Zm+z,:] != 0):
                    finalDeptho[Zfocus+z,:] += currentDeptho[Zm+z,:]/count
        
        finalDeptho = finalDeptho.astype(np.uint16)
        
        fig, ax = plt.subplots(1,1)
        ax.imshow(finalDeptho)
        fig.suptitle(str(size))
        fig.show()
        
        depthoSavePath = os.path.join(savePath, specif + '_M' + str(int(size*100)) + '_step' + str(step) + '_Deptho.tif')
        io.imsave(depthoSavePath, finalDeptho)
        metadataPath = os.path.join(savePath, specif + '_M' + str(int(size*100)) + '_step' + str(step) + '_Metadata.csv')
        with open(metadataPath, 'w') as f:
            f.write('step;focus')
            f.write('\n')
            f.write(str(step) + ';' + str(Zfocus))
        
        print('ok')
               
        
# Finished !

In [86]:
# dirPath = 'C://Users//JosephVermeil//Desktop//test_DepthoMaking'
dirPath = 'D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M1py'
savePath = 'D://MagneticPincherData//Raw//EtalonnageZ'
specif = '21-07-08_M1'
SCALE_100X = 15.8 # pix/µm

depthoMaker(dirPath, savePath, specif, SCALE_100X, beadSize = 'detect', step = 20, d = 'HD', Tmethod = 'otsu', Tfactor = 1.5)

# dirPath = 'D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M2py'
# savePath = 'D://MagneticPincherData//Raw//EtalonnageZ'
# specif = '21-07-08_M2'
# SCALE_100X = 15.8 # pix/µm

# depthoMaker(dirPath, savePath, specif, SCALE_100X, beadSize = 'detect', step = 20, d = 'HD', Tmethod = 'otsu', Tfactor = 1.5)

D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M1py\d1.tif
7972.450000000001
Not acceptable bead in x0, y0 = 687.12, 296.24


Job done for the bead in x0, y0 = 580.27, 329.87




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: divide by zero encountered in double_scalars


Job done for the bead in x0, y0 = 912.32, 365.90


Job done for the bead in x0, y0 = 193.19, 473.22


Job done for the bead in x0, y0 = 123.81, 529.30


D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M1py\d2.tif
8089.300000000001
Job done for the bead in x0, y0 = 708.70, 206.78


Job done for the bead in x0, y0 = 1407.54, 492.63




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: divide by zero encountered in double_scalars


Job done for the bead in x0, y0 = 627.80, 531.27


D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M1py\d3.tif
5587.5
Not acceptable bead in x0, y0 = 9345.38, 2749.69


Not acceptable bead in x0, y0 = 595.77, 108.38


Not acceptable bead in x0, y0 = 278.16, 127.61


Not acceptable bead in x0, y0 = 1490.47, 387.60


Not acceptable bead in x0, y0 = 136.60, 426.79


D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M1py\d4.tif
6037.5
Not acceptable bead in x0, y0 = 1280.87, 119.51


Not acceptable bead in x0, y0 = 1409.29, 122.95


Not acceptable bead in x0, y0 = 53.95, 252.27


Not acceptable bead in x0, y0 = 748.49, 314.53


D://MagneticPincherData//Raw//21.07.08_Deptho//deptho_M1py\d5.tif
6109.5
Not acceptable bead in x0, y0 = 132.62, 266.35


Not acceptable bead in x0, y0 = 135.01, 354.73


ok
ok


In [72]:
# # Test depthoMaker

# dirPath = 'C://Users//JosephVermeil//Desktop//test_DepthoMaking'
dirPath = 'D://MagneticPincherData//Raw//21.09.09_Deptho//M1_py'
savePath = 'D://MagneticPincherData//Raw//EtalonnageZ'
specif = '21-09-09_M1'
SCALE_100X = 15.8 # pix/µm

depthoMaker(dirPath, savePath, specif, SCALE_100X, beadSize = 'M450', step = 20, d = 'HD')

D://MagneticPincherData//Raw//21.09.09_Deptho//M1_py\M1_1.tif
6692.4
6692.4
True 1


C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in double_scalars


Job done for the bead in x0, y0 = 1510.09, 154.03


6692.4
True 1
Job done for the bead in x0, y0 = 387.72, 209.62


6692.4
True 1
Job done for the bead in x0, y0 = 1517.39, 609.78


6692.4
True 1
Job done for the bead in x0, y0 = 629.68, 682.72


D://MagneticPincherData//Raw//21.09.09_Deptho//M1_py\M1_2.tif
6700.8
6700.8
True 1
Job done for the bead in x0, y0 = 681.93, 337.78


6700.8
True 1
Job done for the bead in x0, y0 = 1338.01, 344.58


6700.8
True 1
Job done for the bead in x0, y0 = 1029.44, 374.30


6700.8
True 1
Job done for the bead in x0, y0 = 804.75, 472.58


6700.8
True 1
Job done for the bead in x0, y0 = 1253.36, 616.13


ok


In [17]:
# # Test interpolation
# from scipy import interpolate

# x = np.arange(-5.01, 5.01, 0.25)
# y = np.arange(-5.01, 5.01, 0.25)
# xx, yy = np.meshgrid(x, y)
# z = np.sin(xx**2+yy**2)
# f = interpolate.interp2d(x, y, z, kind='cubic')
# # Now use the obtained interpolation function and plot the result:

# xnew = np.arange(-5.01, 5.01, 1e-3)
# ynew = np.arange(-5.01, 5.01, 1e-3)
# znew = f(xnew, ynew)
# # plt.plot(x, z[10, :], 'ro-')
# # plt.plot(xnew, znew[250, :], 'b-')
# # plt.show()

# fig, axes = plt.subplots(1,2)
# axes[0].imshow(z)
# axes[1].imshow(znew)
# fig.show()

In [18]:
# # Test interpolation 02
# from scipy import interpolate

# vals = [[1083,1201,1170,1178,1238],
# [1053,1180,1109,1120,1228],
# [1005,1124,1081,1136,1080],
# [1031,1089,1068,1058,1132],
# [993,1028,1121,996,1080],
# [1028,965,1071,995,1050],
# [1024,1044,1056,1081,1113],
# [994,1030,984,1083,1086],
# [1040,1083,1076,1066,1094],
# [1016,1032,1078,1050,1026],
# [998,1099,1015,1098,1162],
# [1085,1126,1028,1135,1039],
# [1130,1038,1032,1134,1154],
# [1173,1138,1120,1162,1141],
# [1182,1139,1039,1188,1217],
# [1084,1279,1222,1167,1181],
# [1270,1263,1185,1271,1331],
# [1369,1431,1309,1318,1333],
# [1484,1501,1537,1491,1473],
# [1551,1605,1549,1654,1689],
# [1854,1822,1924,1901,2045],
# [2196,2272,2332,2308,2404],
# [2821,2975,3107,3002,2893],
# [3566,3847,3597,3637,3645],
# [4576,4743,4869,4830,4656],
# [6032,6013,6026,6184,5776],
# [7637,7780,7522,7718,7242],
# [9464,9392,9540,9252,8921],
# [11739,11603,11185,10966,10416],
# [13157,13740,13316,12804,11805],
# [15464,15385,15336,14226,13528],
# [17149,17110,16527,15647,14728],
# [19051,18725,18172,17402,16750],
# [19978,19471,18925,18453,17269],
# [20319,20151,19448,18940,17746],
# [20808,20338,19410,19313,18689],
# [20580,20334,19522,19134,18914],
# [19946,20259,19486,18682,18990],
# [19047,19289,18962,18524,18465],
# [18077,18010,18327,18299,17904],
# [16552,16585,16607,17070,17007],
# [14723,15090,15821,16208,15954],
# [13194,13441,14064,14342,15243],
# [11297,11899,12381,12762,12928],
# [9832,10174,10620,11229,11129],
# [8269,8337,9129,9701,9866],
# [6522,6951,7596,7942,8363],
# [5323,5794,6031,6376,6686],
# [4127,4780,4977,5360,5405],
# [3530,3729,4004,4012,4356],
# [2677,3083,3074,3445,3378],
# [2411,2352,2577,2707,2973],
# [1836,1879,1999,2090,2321],
# [1633,1655,1698,1842,1938],
# [1518,1464,1605,1676,1617],
# [1389,1355,1391,1493,1427],
# [1263,1322,1379,1382,1444],
# [1223,1233,1300,1264,1380],
# [1138,1159,1227,1216,1160],
# [1026,1200,1241,1165,1180],
# [1117,1146,1135,1106,1122],
# [1132,1210,1079,1147,1021],
# [1098,1142,1056,1018,1090],
# [1051,1055,1020,1107,1066],
# [1100,1070,1109,1136,1142],
# [1099,1091,1020,1065,1079],
# [1113,1000,1083,1050,1070],
# [1083,1093,1088,1175,1024],
# [1136,1126,1182,1173,1089],
# [1071,1125,1200,1251,1103],
# [1217,1220,1153,1155,1046],
# [1277,1241,1277,1324,1342],
# [1393,1290,1323,1266,1292]]

# Nframes = 1
# cleanSize = 73
# hdSize = 1001
# listProfiles = np.zeros((Nframes, hdSize))
# z = 0
# xx = np.arange(0, 5)
# yy = np.arange(0, cleanSize)
# figV, axV = plt.subplots(1,2)
# axV[0].imshow(vals)
# f = interpolate.RectBivariateSpline(yy,xx,vals)

# # Now use the obtained interpolation function and plot the result:
# xxnew = xx
# yynew = np.linspace(0, cleanSize, hdSize)
# vals_new = f(yynew,xxnew)
# axV[1].imshow(vals_new)
# listProfiles[z,:] = vals_new[:,5//2] * (1/5)
# for i in range(1, 1 + 5//2):
#     listProfiles[z,:] += vals_new[:,5//2-i] * (1/5)
#     listProfiles[z,:] += vals_new[:,5//2+i] * (1/5)   

In [19]:
# plt.close('all')

In [20]:
# # Test mbd.build_listDepthos
# # testDataFolder = 'C://Users//JosephVermeil//Desktop//test_DepthoMaking'
# testDataFolder = 'C://Users//JosephVermeil//Desktop//test_DepthoMaking'
# listDepthos = [f for f in os.listdir(testDataFolder) if (os.path.isfile(os.path.join(testDataFolder, f)) and f.endswith(".tif"))]
# print(listDepthos)

# f = listDepthos[0]
# filePath = os.path.join(testDataFolder, f)
# I = io.imread(filePath)
# mbd = MultiBeadsDeptho(I, f[:-4], SCALE_100X)
# mbd.build_listDepthos(size = 'detect', plot = 1)
# # listBeads = mbd.get_beadList()

## Main

In [12]:
mainDataDir = 'D://MagneticPincherData'
# mainDataDir = 'C://Users//josep//Desktop//TestData_BeadTracker'
rawDataDir = os.path.join(mainDataDir, 'Raw')
depthoDir = os.path.join(rawDataDir, 'EtalonnageZ')
interDataDir = os.path.join(mainDataDir, 'Intermediate')
figureDir = os.path.join(mainDataDir, 'Figures')
timeSeriesDataDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//DataAnalysis//TimeSeriesData"
dateTest1 = '21-04-27'
dateTest2 = '21.04.27'
# 21-04-27_M1_P1_C4_R40_disc20um_wFluo
# imagesToAnalyse.append(os.path.join(rd, f))

In [13]:
dirD = 'D://MagneticPincherData//Raw//EtalonnageZ'
fileTestPath = os.path.join(dirD, '21-09-09_Deptho_M2.tif')
Dtest = io.imread(fileTestPath)
Dtest.shape

(372, 1001)

In [14]:
def main(dates, manips, wells, cells, depthoNames, redoAllSteps = False):
    
    start = time.time()
    
    # 0. Load different data sources & Preprocess : fluo, black images, sort slices (ct/ramp ; down/middle/up)
    # Make list of files to analyse
    imagesToAnalyse = []
    imagesToAnalyse_Paths = []
    if not isinstance(dates, str):
        rawDirList = [os.path.join(rawDataDir, d) for d in dates]
    else:
        rawDirList = [os.path.join(rawDataDir, dates)]
    for rd in rawDirList:
        fileList = os.listdir(rd)
        for f in fileList:
            if isFileOfInterest(f, manips, wells, cells):
                imagesToAnalyse.append(f)
                imagesToAnalyse_Paths.append(os.path.join(rd, f))          
    
    # Begining of the MAIN LOOP
        
    for i in range(len(imagesToAnalyse)): 
        f, fP = imagesToAnalyse[i], imagesToAnalyse_Paths[i]
        manipID = findInfosInFileName(f, 'manipID')
        cellID = findInfosInFileName(f, 'cellID')
        
        print('\n')
        print('Analysis of file {:.0f}/{:.0f} : {}'.format(i+1, len(imagesToAnalyse), f))
        print('Loading image and experimental data...')
        
        # Load exp data
        if manipID not in expDf['manipID'].values:
            print('Error! No experimental data found for: ' + manipID)
            bug
        else:
            expDf_line = expDf.loc[expDf['manipID'] == manipID]
            manipDict = {}
            for c in expDf_line.columns.values:
                manipDict[c] = expDf_line[c].values[0]
        
#         # Extract some data from exp data
#         StrBeadTypes = str(manipDict['bead type'])
#         if '_' in StrBeadTypes:
#             beadTypes = [bT for bT in StrBeadTypes.split('_')]
#         else:
#             beadTypes = [StrBeadTypes]
        
#         StrBeadDiameters = str(manipDict['bead diameter'])
#         if '_' in StrBeadDiameters:
#             beadDiameters = [int(bD) for bD in StrBeadDiameters.split('_')]
#         else:
#             beadDiameters = [int(StrBeadDiameters)]
            
#         dictBeadDiameters = {}
#         for k in range(len(beadTypes)):
#             dictBeadDiameters[beadTypes[k]] = beadDiameters[k]
    
    
        # Load image and init PTL
        I = io.imread(fP) # Approx 0.5s per image
        PTL = PincherTimeLapse(I, cellID, manipDict, NB = 2)
    
        # Load field file
        fieldFilePath = fP[:-4] + '_Field.txt'
        fieldCols = ['B_set', 'T_abs', 'B', 'Z']
        fieldDf = pd.read_csv(fieldFilePath, sep = '\t', names = fieldCols)
        
        # Check if a log file exists and load it if required
        logFilePath = fP[:-4] + '_LogPY.txt'
        logFileImported = False
        if redoAllSteps:
            pass
        elif os.path.isfile(logFilePath):
            PTL.importLog(logFilePath)
            PTL.dictLog['UILog'] = PTL.dictLog['UILog'].astype(str)
            logFileImported = True
            
        print('OK!')
        
        print('Pretreating the image...')
        
        # Detect fluo & black images
        PTL.checkIfBlackFrames()
        PTL.saveFluoAside()
        if not logFileImported:
            pass # I may change the organization of this part later
        
        # Sort slices
        if not logFileImported:
            if 'R40' or 'thickness5mT' in f:
                PTL.determineFramesStatus_R40()
        PTL.saveLog(display = False, save = (not logFileImported), path = logFilePath)
        
        # Import or determine global threshold
        MDpath = fP[:-4] + '_MetaDataPY.txt'
        if redoAllSteps:
            PTL.uiThresholding()
        else:
            try:
                PTL.threshold = PTL.readMetaData(MDpath, 'threshold')
            except:
                PTL.uiThresholding() # Approx 3s per image
#         PTL.testThresholding()

        # Save some metadata
        PTL.saveMetaData(MDpath)
        
        # Create list of Frame objects
        PTL.makeFramesList()
        
        print('OK!')
        
    # 1. Detect beads
    
        print('Detecting all the bead objects...')
        Td = time.time()
        
        # Check if a _Results.txt exists and import it if so
        resFilePath = fP[:-4] + '_ResultsPY.txt'
        resFileImported = False
        if redoAllSteps:
            pass
        elif os.path.isfile(resFilePath):
            PTL.importBeadsDetectResult(resFilePath)
            resFileImported = True
    
        # Dectect the beads and create the BeadsDetectResult dataframe [if no file has been loaded before] 
        # OR input the results in each Frame objects [if the results have been loaded at the previous step]
        PTL.detectBeads(resFileImported, display = 0)
        
        # Save the new results if necessary
        if not resFileImported:
            PTL.saveBeadsDetectResult(path=resFilePath)
            
        print('OK! dT = {:.3f}'.format(time.time()-Td))

    # 2. Make trajectories for beads of interest
    
        print('Tracking the beads of interest...')
        Tt = time.time()
        
        # One of the main steps ! The tracking of the beads happens here !
        issue = PTL.buildTrajectories() 
        if issue == 'Bug':
            continue
        else:
            pass
        
        # Save the user inputs
        PTL.saveLog(display = 0, save = True, path = logFilePath)
        
        print('OK! dT = {:.3f}'.format(time.time()-Tt))
    
    # 3. Qualify - Detect boi sizes and neighbours
        # Detect or infer Boi sizes in the first image
        if len(PTL.beadTypes) == 1:
            if 'M450' in PTL.beadTypes[0]:
                D = 4.5
            elif 'M270' in PTL.beadTypes[0]:
                D = 2.7
            for B in PTL.listFrames[0].listBeads:
                B.D = D
        else:
            PTL.listFrames[0].detectDiameter(plot = 0)
            
        # Propagate it across the trajectories
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            B0 = traj.dict['Bead'][0]
            D = B0.D
            traj.D = D
            for B in traj.dict['Bead']:
                B.D = D
        
        # Detect neighbours in every first image of loop TBC !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # !!!!!!
        
#         iL = 0 # index to count the loops
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            beadType = ''
            if len(PTL.beadTypes) == 1:
                beadType = PTL.beadTypes[0] # M270 or M450
            else:
                beadType = 'detect'
            traj.detectNeighbours(frequency = PTL.loop_totalSize, beadType = beadType)
            

    
        
    # 4. Compute dz
        # Import depthographs
        HDZfactor = 10
        
        if len(PTL.beadTypes) == 1:
            depthoPath = os.path.join(depthoDir, depthoNames)
#             depthoExist = os.path.exists(depthoPath+'_Deptho.tif')
            deptho = io.imread(depthoPath+'_Deptho.tif')
            depthoMetadata = pd.read_csv(depthoPath+'_Metadata.csv', sep=';')
            depthoStep = depthoMetadata.loc[0,'step']
            depthoZFocus = depthoMetadata.loc[0,'focus']
            
            # increase the resolution of the deptho with interpolation
            nX, nZ = deptho.shape[1], deptho.shape[0]
            XX, ZZ = np.arange(0, nX, 1), np.arange(0, nZ, 1)
            fd = interpolate.interp2d(XX, ZZ, deptho, kind='cubic')
            ZZ_HD = np.arange(0, nZ, 1/HDZfactor)
            depthoHD = fd(XX, ZZ_HD)
            depthoStepHD = depthoStep/HDZfactor
            depthoZFocus = depthoZFocus*HDZfactor
            #
            for iB in range(PTL.NB):
                traj = PTL.listTrajectories[iB]
                traj.deptho = depthoHD
                traj.depthoPath = depthoPath
                traj.depthoStep = depthoStepHD
                traj.depthoZFocus = depthoZFocus
        
        if len(PTL.beadTypes) > 1:
            for dN in depthoNames:
                depthoPath = os.path.join(depthoDir, dN)
                deptho = io.imread(depthoPath+'_Deptho.tif')
                depthoMetadata = pd.read_csv(depthoPath+'_Metadata.csv', sep=';')
                depthoStep = depthoMetadata.loc[0,'step']
                depthoZFocus = depthoMetadata.loc[0,'focus']
                
                # increase the resolution of the deptho with interpolation
                nX, nZ = deptho.shape[1], deptho.shape[0]
                XX, ZZ = np.arange(0, nX, 1), np.arange(0, nZ, 1)
                fd = interpolate.interp2d(XX, ZZ, deptho, kind='cubic')
                ZZ_HD = np.arange(0, nZ, 1/HDZfactor)
                depthoHD = fd(XX, ZZ_HD)
                depthoStepHD = depthoStep/HDZfactor
                depthoZFocusHD = depthoZFocus*HDZfactor
                #
                if 'M450' in dN:
                    depthoD = 4.5
                elif 'M270' in dN:
                    depthoD = 2.7
                for iB in range(PTL.NB):
                    traj = PTL.listTrajectories[iB]
                    if traj.D == depthoD:
                        traj.deptho = depthoHD
                        traj.depthoPath = depthoPath
                        traj.depthoStep = depthoStepHD
                        traj.depthoZFocus = depthoZFocusHD
            
        # Compute z for each traj
        for iB in range(PTL.NB):
            np.set_printoptions(threshold=np.inf)
            
            print('Computing Z in traj  {:.0f}...'.format(iB+1))
            Tz = time.time()
            traj = PTL.listTrajectories[iB]
            traj.computeZ(plot = 0)
            print('OK! dT = {:.3f}'.format(time.time()-Tz))
            
        # Keep only the best std data in the trajectories
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            traj.keepBestStdOnly()
    
    
    # 5. Define pairs and compute dx, dy
        
        print('Computing distances...')
        if PTL.NB == 2:
            traj1 = PTL.listTrajectories[0]
            traj2 = PTL.listTrajectories[1]
            nT = traj1.nT
            
            # Create a dict to prepare the export of the results
            timeSeries = {
                'idxCompression' : np.zeros(nT),
                'T' : np.zeros(nT),
                'Tabs' : np.zeros(nT),
                'B' : np.zeros(nT),
                'F' : np.zeros(nT),
                'dx' : np.zeros(nT),
                'dy' : np.zeros(nT),
                'dz' : np.zeros(nT),
                'D2' : np.zeros(nT),
                'D3' : np.zeros(nT)
            }

            # Input common values:
            T0 = fieldDf['T_abs'].values[0]/1000
            timeSeries['idxCompression'] = traj1.dict['idxCompression']
            timeSeries['Tabs'] = (fieldDf['T_abs'][traj1.dict['iField']])/1000
            timeSeries['T'] = timeSeries['Tabs'].values - T0*np.ones(nT)
            timeSeries['B'] = fieldDf['B_set'][traj1.dict['iField']].values
            timeSeries['B'] *= PTL.MagCorrFactor

            # Compute distances
            timeSeries['dx'] = (traj2.dict['X'] - traj1.dict['X'])/PTL.scale
            timeSeries['dy'] = (traj2.dict['Y'] - traj1.dict['Y'])/PTL.scale
            timeSeries['D2'] = (timeSeries['dx']**2 +  timeSeries['dy']**2)**0.5
            
            timeSeries['dz'] = (traj2.dict['Zr']*traj2.depthoStep - traj1.dict['Zr']*traj1.depthoStep)/1000
            timeSeries['dz'] *= PTL.OptCorrFactor
            timeSeries['D3'] = (timeSeries['D2']**2 +  timeSeries['dz']**2)**0.5
            
            
            #print('\n\n* timeSeries:\n')
            #print(timeSeries_DF[['T','B','F','dx','dy','dz','D2','D3']])
            print('OK!')
            
    # 6. Compute forces
        print('Computing forces...')
        Tf = time.time()
        if PTL.NB == 2:
            traj1 = PTL.listTrajectories[0]
            traj2 = PTL.listTrajectories[1]
            B0 = timeSeries['B']
            D3 = timeSeries['D3']
            dx = timeSeries['dx']
            F, dfLogF = PTL.forceCompute(traj1, traj2, B0, D3, dx)
            timeSeries['F'] = F
        
        print('OK! dT = {:.3f}'.format(time.time()-Tf))
            
            # Magnetization [A.m^-1]
            # M270
            # M = 0.74257*1.05*1600*(0.001991*B.^3+17.54*B.^2+153.4*B)./(B.^2+35.53*B+158.1)
            # M450
            # M = 1.05*1600*(0.001991*B.^3+17.54*B.^2+153.4*B)./(B.^2+35.53*B+158.1);
    
    # Save the tables !
        if PTL.NB == 2:
            timeSeries_DF = pd.DataFrame(timeSeries)
            timeSeriesFilePath = os.path.join(timeSeriesDataDir, f[:-4] + '_PY.csv')
            timeSeries_DF.to_csv(timeSeriesFilePath, sep = ';', index=False)
    
    
          
    # 7. Export the results
    
    print('\nTotal time:')
    print(time.time()-start)
    print('\n')
    return(PTL, timeSeries_DF, dfLogF)

#

In [15]:
def forceCompute(traj1, traj2, B0, D3, dx, dictBeadDiameters):
    
    # Magnetization functions
    
    def computeMag_M270(B):
        M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
        return(M)
    
    def computeMag_M450(B):
        M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
        return(M)
    
    dictMag = {'M270' : computeMag_M270, 'M450' : computeMag_M450}
    dictBeadTypes = {2.7 : 'M270', 4.5 : 'M450'}
    
    # Correction functions
    
    def Bind_neighbour(B, D_BoI, neighbourType):
        if neighbourType == '':
            return(0)
        
        else:
            D_neighbour = dictBeadDiameters[neighbourType]
            f_Mag = dictMag[neighbourType] # Appropriate magnetization function
            M_neighbour = f_Mag(B) # magnetization [A.m^-1]
            V_neighbour = (4/3)*np.pi*(D_neighbour/2)**3 # volume [nm^3]
            m_neighbour = M_neighbour*V_neighbour*1e-9 # magnetic moment [A.nm^2]

            D_tot = (D_BoI + D_neighbour)/2 # Center-to-center distance [nm]
            B_ind = 2e5*m_neighbour/(D_tot**3) # Inducted mag field [mT]
            return(B_ind)

    def deltaF_neighbour(m_BoI, B, D_BoI, D_BoI2, neighbourType):
        if neighbourType == '':
            return(0)
        
        else:
            D_neighbour = dictBeadDiameters[neighbourType]
            f_Mag = dictMag[neighbourType] # Appropriate magnetization function
            M_neighbour = f_Mag(B) # magnetization [A.m^-1]
            V_neighbour = (4/3)*np.pi*(D_neighbour/2)**3 # volume [nm^3]
            m_neighbour = M_neighbour*V_neighbour*1e-9 # magnetic moment [A.nm^2]

            D_tot = D_BoI/2 + D_BoI2 + D_neighbour/2
            deltaF = 3e5*m_BoI*m_neighbour/Dtot**4 # force [pN]
            return(deltaF)
        
    
    
    # Let's make sure traj1 is the left bead traj and traj2 the right one.
    avgX1 = np.mean(traj1.dict.X)
    avgX2 = np.mean(traj2.dict.X)
    if avgX1 < avgX2:
        traj_L, traj_R = traj1, traj2
    else:
        traj_L, traj_R = traj2, traj1
    
    # Get useful data
    BeadType_L, BeadType_R = dictBeadTypes[traj_L.D], dictBeadTypes[traj_R.D]
    Neighbours_BL = np.concatenate(([traj_L.Neighbour_L], [traj_L.Neighbour_R]), axis = 0)
    Neighbours_BR = np.concatenate(([traj_R.Neighbour_L], [traj_R.Neighbour_R]), axis = 0)
    D_L, D_R = dictBeadDiameters[BeadType_L], dictBeadDiameters[BeadType_R]
    
    nT = len(B0)
    D3nm = 1000*D3
    Dxnm = 1000*dx
    F = np.zeros(nT)
    
    # Maybe possible to process that faster on lists themselves
    for i in range(nT):
        # Appropriate magnetization functions
        f_Mag_L = dictMag[BeadType_L]
        f_Mag_R = dictMag[BeadType_R]
        
        # Btot = B0 + B inducted by potential left neighbour mag + B inducted by potential right neighbour mag
        Btot_L = B0[i] + Bind_neighbour(B0[i], D_L, Neighbours_BL[0,i]) + Bind_neighbour(B0[i], D_L, Neighbours_BL[1,i])
        Btot_R = B0[i] + Bind_neighbour(B0[i], D_R, Neighbours_BR[0,i]) + Bind_neighbour(B0[i], D_R, Neighbours_BR[1,i])
        
        # Magnetizations
        M_L = f_Mag_L(Btot_L)
        M_R = f_Mag_R(Btot_R)
        
        # Volumes
        V_L = (4/3)*np.pi*(D_L/2)**3 # volume [nm^3]
        V_R = (4/3)*np.pi*(D_R/2)**3 # volume [nm^3]
        
        # Magnetizations
        m_L = M_L * 1e-9 * V_L
        m_R = M_R * 1e-9 * V_R
        
        anglefactor = abs(3*(dxnm[i]/D3nm[i])**2 - 1)
        
        # Forces
        F0 = 3e5*anglefactor*m_L*m_R/D3nm[i]**4
        dF_L = deltaF_neighbour(m_L, B0[i], D_L, D_R, Neighbours_BR[1,i])
        dF_R = deltaF_neighbour(m_R, B0[i], D_R, D_L, Neighbours_BL[0,i])
        
        # Total force = force between beads involved in the pair (F0)
        #               + small force between B_L and B_R's potential right neighbour
        #               + small force between B_R and B_L's potential left neighbour
        F[i] = F0 + dF_L + dF_R
        
        

In [16]:
#     dates = '21.01.21'
#     manips = 1
#     wells = 1
#     cells = 1
#     depthoNames = '21-01-21_M1_M450_step20'
    
dates = '21.09.09'
manips = 1
wells = 1
cells = [3]
depthoNames = '21-09-09_M1_M450_step20'

PTL, timeSeries_DF, dfLogF = main(dates, manips, wells, cells, depthoNames, redoAllSteps = False)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'




Analysis of file 1/1 : 21-09-09_M1_P1_C3_R40_disc20um.tif
Loading image and experimental data...
OK!
Pretreating the image...
OK!
Detecting all the bead objects...
OK! dT = 1.324
Tracking the beads of interest...
Error for 1680
Error for 1686
Error for 1689
OK! dT = 0.906


C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:217: RuntimeWarning: divide by zero encountered in long_scalars


Computing Z in traj  1...
OK! dT = 19.982
Computing Z in traj  2...
OK! dT = 19.709
Computing distances...
OK!
Computing forces...
OK! dT = 0.041

Total time:
47.690513610839844




In [42]:
pd.set_option('display.max_rows', None)
dfLogF_full = dfLogF.loc[0:1,['D3','B0','Btot_L','F00','F0','Ftot']]
dfLogF_full = dfLogF_full.rename(columns = {'D3':'D3 (nm)','B0':'B0 (mT)','Btot_L':'B_tot (mT)','F00':'F_noNeighbours (pN)','F0':'F_BCorrected (pN)','Ftot':'F_tot (pN)'})
dfLogF_full_s = dfLogF_full.style.format('{:.1f}')

dfLogF_full_s

,D3 (nm),B0 (mT),B_tot (mT),F_noNeighbours (pN),F_BCorrected (pN),F_tot (pN)
0,976.8,5.8,7.1,56.7,75.4,84.7
1,987.1,5.8,7.1,56.2,74.8,84.1


In [43]:
dfLogF_light = dfLogF.loc[0:1,['B0','F00','F0','Ftot']]
dfLogF_light = dfLogF_light.rename(columns = {'D3':'D3 (nm)','B0':'B0 (mT)','Btot_L':'B_tot (mT)','F00':'F_noNeighbours (pN)','F0':'F_BCorrected (pN)','Ftot':'F_tot (pN)'})
dfLogF_light_s = dfLogF_light.style.format('{:.1f}')

dfLogF_light_s

,B0 (mT),F_noNeighbours (pN),F_BCorrected (pN),F_tot (pN)
0,5.8,56.7,75.4,84.7
1,5.8,56.2,74.8,84.1


In [ ]:
traj = PTL.listTrajectories[0]
dic = {}
liste = ['X','Y','idxCompression','StdDev', 
                     'Bead','Status','Status_2','iF','iS','iB',
                     'bestStd']
for k in liste:
    dic[k] = traj.dict[k]
df1 = pd.DataFrame(dic)
df1

In [29]:
fig, ax = plt.subplots(1,1)

t = timeSeries_DF['T']
dz = timeSeries_DF['dz']
d2 = timeSeries_DF['D2']-4.503
d3 = timeSeries_DF['D3']-4.503

ax.plot(t,dz,'-b')
ax.plot(t,d3,'-r')

fig.show()

In [30]:
PTL.threshold

8770.95

In [37]:
frame = PTL.listFrames[-5].show()
ax = plt.gca()
for i in range(PTL.NB):
    PTL.listTrajectories[i].plot(ax, i)
plt.show()

In [31]:
# traj = PTL.listTrajectories[0]
# traj.computeZ(plot = 1)
# d = traj.deptho
# # plt.imshow(d)
# print(traj.depthoStep)

In [35]:
traj = PTL.listTrajectories[1]
traj.dict['Neighbour_L']

array(['M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
       'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450', 'M450',
      

In [36]:
traj = PTL.listTrajectories[1]
traj.computeZ(plot = 1)
d = traj.deptho
# plt.imshow(d)

In [140]:
np.set_printoptions(threshold=np.inf)
traj = PTL.listTrajectories[0]
listeNup = np.array([traj.listFrames[i].Nuplet for i in traj.dict['iF']])
listeNup

In [ ]:
PTL.listTrajectories[0].to_df()

In [ ]:
traj = PTL.listTrajectories[1]
a = np.sum(np.array(traj.dict['idxCompression']) == 1)
traj.dict['bestStd'][:]

In [ ]:
traj = PTL.listTrajectories[1]
traj.dict['iField'][:]

In [ ]:
traj = PTL.listTrajectories[1]
np.array(traj.dict['idxRamp']) == 1

In [ ]:
# PTL.dictLog['UILog'] = PTL.dictLog['UILog'].astype(str)
a = PTL.dictLog['UILog'].astype(str)
a

In [185]:
plt.close('all')

In [ ]:
# fp = 'D://MagneticPincherData//Raw//21.04.27//21-04-27_M1_P1_C4_R40_disc20um_wFluo.tif'
# fp = "C://Users//JosephVermeil//Desktop//Fichier 3BiocheVsPhy@2x.png"
# fp = "C://Users//JosephVermeil//Desktop//21-04-27_M1_P1_C4_R40_disc20um_wFluo.tif"
# I = io.imread(fp)
# I

In [ ]:
a = ['a']
b = 'a'
len(a[0][0])
len(b[0][0])
isinstance(b, str)

In [ ]:
s = 'Analysis of file {:.0f}/{:.0f} : {}'.format(1, 20, 'Aaa')
print(s)

In [ ]:
dictD = {2.7 : 'M270', 4.5 : 'M450'}
dictD[4.5]

In [ ]:
A = np.array(['a', 'b', 'e'])
B = np.array(['c', 'd', 'f'])
C = np.concatenate(([A], [B]), axis = 0)
C[0,:]
C[1,:]

In [45]:
a, b = 1, 2
a, b = b, a
print(a, b)

2 1


In [ ]:
# def testR():
#     def f1(x):
#         return(2*x)

#     def f2(x):
#         return(x**2 + 6)
    
#     F = [f1, f2]
#     L = [1, 2]
#     for i in range(len(L)):
#         f = F[i]
#         print(f(L[i]))
        
# testR()

### Old code

In [29]:
# Previously used to detect neighbours

#             neighbours = np.array([['',''] for i in range(traj.nT)], dtype = '<U12')
#             current_neighbours = ['','']
#             for i in range(traj.nT):
#                 iS = traj.dict['iS'][i]
#                 if iS >= iL*PTL.loop_totalSize:
#                     # Redo the detection
#                     iF = traj.dict['iF'][i]
#                     iBead = traj.dict['iB'][i]
#                     frame = PTL.listFrames[iF]
#                     current_neighbours = frame.detectNeighbours(iBead)
#                     neighbours[i] = current_neighbours
#                     #
#                     iL += 1
#                 else:
#                     # Just take the previous neighbour situation
#                     neighbours[i] = current_neighbours
#             traj.dict['Neighbours'] = neighbours